© 2026 Nokia
          Licensed under the BSD 3-Clause Clear License
          SPDX-License-Identifier: BSD-3-Clause-Clear

In [ ]:
!pip install seaborn

In [ ]:
import time
import pandas as pd
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
# configure: host name, nic, cores, ram, disk according to your needs.

slice_name="wan-inter-site-large-scenario" + fablib.get_bastion_username()

node_conf = [
 {'name': "sender1",   'host': 'ncsa-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender2",   'host': 'ncsa-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender3",   'host': 'ncsa-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender4",   'host': 'ncsa-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},


 {'name': "inter-router1",   'host': 'ncsa-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "inter-router2",   'host': 'ncsa-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "NCSA", 'cores': 16, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},


 {'name': "sender5",   'host': 'mich-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender6",   'host': 'mich-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender7",   'host': 'mich-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "sender8",   'host': 'mich-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 12, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},

 {'name': "inter-router3",   'host': 'mich-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "inter-router4",   'host': 'mich-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "MICH", 'cores': 16, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},


 {'name': "router1",   'host': 'ncsa-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_6', 'site_name': "NCSA", 'cores': 64, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "router2",   'host': 'mich-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_6', 'site_name': "MICH", 'cores': 64, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "router3",   'host': 'kans-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_6', 'site_name': "KANS", 'cores': 64, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "router4",   'host': 'indi-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_6', 'site_name': "INDI", 'cores': 64, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},


 {'name': "receiver1", 'host': 'kans-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "KANS", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver2", 'host': 'kans-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "KANS", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver3", 'host': 'indi-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "INDI", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver4", 'host': 'indi-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "INDI", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},

 {'name': "receiver5", 'host': 'kans-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "KANS", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver6", 'host': 'kans-w3.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "KANS", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver7", 'host': 'indi-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "INDI", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "receiver8", 'host': 'indi-w1.fabric-testbed.net', 'nic': 'NIC_Basic', 'site_name': "INDI", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}

]

net_conf = [

 {"name": "net1-sender-intermediate-router-1", "subnet": "10.0.1.0/24", "nodes": [{"name": "sender1",   "addr": "10.0.1.140", 'nic': 'NIC_Basic'}, {"name": "sender2",   "addr": "10.0.1.150", 'nic': 'NIC_Basic'}, {"name": "inter-router1", "addr": "10.0.1.102", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net1-sender-intermediate-router-2", "subnet": "10.0.14.0/24", "nodes": [{"name": "sender3",   "addr": "10.0.14.120", 'nic': 'NIC_Basic'}, {"name": "sender4",   "addr": "10.0.14.130", 'nic': 'NIC_Basic'}, {"name": "inter-router2", "addr": "10.0.14.102", 'nic': 'NIC_ConnectX_6'}]},

 {"name": "intermediate-router1", "subnet": "10.0.13.0/24", "nodes": [{"name": "inter-router1", "addr": "10.0.13.102", 'nic': 'NIC_ConnectX_6'},  {"name": "inter-router2", "addr": "10.0.13.103", 'nic': 'NIC_ConnectX_6'}, {"name": "router1", "addr": "10.0.13.101", 'nic': 'NIC_ConnectX_6'}]},


 {"name": "net2-sender-intermediate-router-1", "subnet": "10.0.2.0/24", "nodes": [{"name": "sender5",   "addr": "10.0.2.140", 'nic': 'NIC_Basic'}, {"name": "sender6",   "addr": "10.0.2.150", 'nic': 'NIC_Basic'}, {"name": "inter-router3", "addr": "10.0.2.102", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-sender-intermediate-router-2", "subnet": "10.0.15.0/24", "nodes": [{"name": "sender7",   "addr": "10.0.15.120", 'nic': 'NIC_Basic'}, {"name": "sender8",   "addr": "10.0.15.130", 'nic': 'NIC_Basic'}, {"name": "inter-router4", "addr": "10.0.15.102", 'nic': 'NIC_ConnectX_6'}]},

 {"name": "intermediate-router2", "subnet": "10.0.16.0/24", "nodes": [{"name": "inter-router3", "addr": "10.0.16.102", 'nic': 'NIC_ConnectX_6'},  {"name": "inter-router4", "addr": "10.0.16.103", 'nic': 'NIC_ConnectX_6'}, {"name": "router2", "addr": "10.0.16.101", 'nic': 'NIC_ConnectX_6'}]},


 {"name": "net1-router1-router3", "vlan": 100, "hops":"STAR", "bw": 20, "subnet": "10.0.3.0/24", "nodes": [{"name": "router1",   "addr": "10.0.3.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router3", "addr": "10.0.3.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-router1-router3", "vlan": 200, "hops":"STAR", "bw": 20, "subnet": "10.0.4.0/24", "nodes": [{"name": "router1",   "addr": "10.0.4.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router3", "addr": "10.0.4.101", 'nic': 'NIC_ConnectX_6'}]},

 {"name": "net1-router1-router4", "vlan": 300, "hops":"STAR", "bw": 20, "subnet": "10.0.5.0/24", "nodes": [{"name": "router1",   "addr": "10.0.5.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router4", "addr": "10.0.5.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-router1-router4", "vlan": 400, "hops":"STAR", "bw": 20, "subnet": "10.0.6.0/24", "nodes": [{"name": "router1",   "addr": "10.0.6.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router4", "addr": "10.0.6.101", 'nic': 'NIC_ConnectX_6'}]},


 {"name": "net1-router2-router3", "vlan": 500, "hops":"STAR", "bw": 20, "subnet": "10.0.7.0/24", "nodes": [{"name": "router2",   "addr": "10.0.7.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router3", "addr": "10.0.7.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-router2-router3", "vlan": 600, "hops":"STAR", "bw": 20, "subnet": "10.0.8.0/24", "nodes": [{"name": "router2",   "addr": "10.0.8.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router3", "addr": "10.0.8.101", 'nic': 'NIC_ConnectX_6'}]},

 {"name": "net1-router2-router4", "vlan": 700, "hops":"STAR", "bw": 20, "subnet": "10.0.9.0/24", "nodes": [{"name": "router2",   "addr": "10.0.9.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router4", "addr": "10.0.9.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-router2-router4", "vlan": 800, "hops":"STAR", "bw": 20, "subnet": "10.0.10.0/24", "nodes": [{"name": "router2",   "addr": "10.0.10.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router4", "addr": "10.0.10.101", 'nic': 'NIC_ConnectX_6'}]},

 {"name": "net1-receiver", "subnet": "10.0.11.0/24", "nodes": [{"name": "receiver1",   "addr": "10.0.11.100", 'nic': 'NIC_Basic'}, {"name": "receiver2",   "addr": "10.0.11.110", 'nic': 'NIC_Basic'}, {"name": "receiver5",   "addr": "10.0.11.120", 'nic': 'NIC_Basic'}, {"name": "receiver6",   "addr": "10.0.11.130", 'nic': 'NIC_Basic'}, {"name": "router3", "addr": "10.0.11.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net2-receiver", "subnet": "10.0.12.0/24", "nodes": [{"name": "receiver7",   "addr": "10.0.12.120", 'nic': 'NIC_Basic'}, {"name": "receiver8",   "addr": "10.0.12.130", 'nic': 'NIC_Basic'}, {"name": "receiver3",   "addr": "10.0.12.140", 'nic': 'NIC_Basic'}, {"name": "receiver4",   "addr": "10.0.12.150", 'nic': 'NIC_Basic'}, {"name": "router4", "addr": "10.0.12.101", 'nic': 'NIC_ConnectX_6'}]}

]


route_conf = [
 {"addr": "10.0.11.0/24", "gw": "10.0.1.102", "nodes": ["sender1","sender2"]},
 {"addr": "10.0.12.0/24", "gw": "10.0.14.102", "nodes": ["sender3","sender4"]},

 {"addr": "10.0.11.0/24", "gw": "10.0.2.102", "nodes": ["sender5","sender6"]},
 {"addr": "10.0.12.0/24", "gw": "10.0.15.102", "nodes": ["sender7","sender8"]},


 {"addr": "10.0.11.0/24", "gw": "10.0.13.101", "nodes": ["inter-router1"]},
 {"addr": "10.0.12.0/24", "gw": "10.0.13.101", "nodes": ["inter-router2"]},


 {"addr": "10.0.11.0/24", "gw": "10.0.16.101", "nodes": ["inter-router3"]},
 {"addr": "10.0.12.0/24", "gw": "10.0.16.101", "nodes": ["inter-router4"]},


 {"addr": "10.0.1.0/24", "gw": "10.0.13.102", "nodes": ["router1"]},
 {"addr": "10.0.14.0/24", "gw": "10.0.13.103", "nodes": ["router1"]},

 {"addr": "10.0.11.100/32", "gw": "10.0.3.101", "nodes": ["router1"]},
 {"addr": "10.0.11.110/32", "gw": "10.0.4.101", "nodes": ["router1"]},

 {"addr": "10.0.12.140/32", "gw": "10.0.5.101", "nodes": ["router1"]},
 {"addr": "10.0.12.150/32", "gw": "10.0.6.101", "nodes": ["router1"]},

 {"addr": "10.0.2.0/24", "gw": "10.0.16.102", "nodes": ["router2"]},
 {"addr": "10.0.15.0/24", "gw": "10.0.16.103", "nodes": ["router2"]},

 {"addr": "10.0.11.120/32", "gw": "10.0.7.101", "nodes": ["router2"]},
 {"addr": "10.0.11.130/32", "gw": "10.0.8.101", "nodes": ["router2"]},

 {"addr": "10.0.12.120/32", "gw": "10.0.9.101", "nodes": ["router2"]},
 {"addr": "10.0.12.130/32", "gw": "10.0.10.101", "nodes": ["router2"]},

 {"addr": "10.0.1.140/32", "gw": "10.0.3.100", "nodes": ["router3"]},
 {"addr": "10.0.1.150/32", "gw": "10.0.4.100", "nodes": ["router3"]},

 {"addr": "10.0.2.140/32", "gw": "10.0.7.100", "nodes": ["router3"]},
 {"addr": "10.0.2.150/32", "gw": "10.0.8.100", "nodes": ["router3"]},

 {"addr": "10.0.14.120/32", "gw": "10.0.5.100", "nodes": ["router4"]},
 {"addr": "10.0.14.130/32", "gw": "10.0.6.100", "nodes": ["router4"]},

 {"addr": "10.0.15.120/32", "gw": "10.0.9.100", "nodes": ["router4"]},
 {"addr": "10.0.15.130/32", "gw": "10.0.10.100", "nodes": ["router4"]},


 {"addr": "10.0.1.0/24", "gw": "10.0.11.101", "nodes": ["receiver1","receiver2"]},
 {"addr": "10.0.14.0/24", "gw": "10.0.12.101", "nodes": ["receiver3","receiver4"]},


 {"addr": "10.0.2.0/24", "gw": "10.0.11.101", "nodes": ["receiver5","receiver6"]},
 {"addr": "10.0.15.0/24", "gw": "10.0.12.101", "nodes": ["receiver7","receiver8"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

Then we will add hosts and network segments:

In [ ]:
# 1) Add nodes + NICs as declared
for n in node_conf:
    node = slice.add_node(
        name=n['name'], site=n['site_name'],
        cores=n['cores'], ram=n['ram'], disk=n['disk'], host=n['host'],
        image=n['image'])
    node.add_component(model=n['nic'], name="nic1")

    if n['name']=="inter-router1" or n['name']=="inter-router2" or n['name']=="inter-router3" or n['name']=="inter-router4":
      node.add_component(model=n['nic'], name="nic2")
      print("added")

In [ ]:
sender1   = slice.get_node("sender1")
sender2  = slice.get_node("sender2")
sender3  = slice.get_node("sender3")
sender4  = slice.get_node("sender4")

sender5   = slice.get_node("sender5")
sender6  = slice.get_node("sender6")
sender7  = slice.get_node("sender7")
sender8  = slice.get_node("sender8")

router1   = slice.get_node("router1")
router2  = slice.get_node("router2")
router3   = slice.get_node("router3")
router4   = slice.get_node("router4")

inter_router1  = slice.get_node("inter-router1")
inter_router2  = slice.get_node("inter-router2")
inter_router3  = slice.get_node("inter-router3")
inter_router4  = slice.get_node("inter-router4")

receiver1 = slice.get_node("receiver1")
receiver2 = slice.get_node("receiver2")
receiver3 = slice.get_node("receiver3")
receiver4 = slice.get_node("receiver4")

receiver5 = slice.get_node("receiver5")
receiver6 = slice.get_node("receiver6")
receiver7 = slice.get_node("receiver7")
receiver8 = slice.get_node("receiver8")

In [ ]:
# # # Shortcuts
# receiver1_ifaces = receiver1.get_component("nic1").get_interfaces()
# receiver2_ifaces = receiver2.get_component("nic1").get_interfaces()
# receiver3_ifaces = receiver3.get_component("nic1").get_interfaces()
# receiver4_ifaces = receiver4.get_component("nic1").get_interfaces()
# receiver5_ifaces = receiver5.get_component("nic1").get_interfaces()
# receiver6_ifaces = receiver6.get_component("nic1").get_interfaces()
# receiver7_ifaces = receiver7.get_component("nic1").get_interfaces()
# receiver8_ifaces = receiver8.get_component("nic1").get_interfaces()

In [ ]:
router1_ifaces   = router1.get_component("nic1").get_interfaces()     # CX6 (2 ports)
router2_ifaces   = router2.get_component("nic1").get_interfaces()     # CX6 (2 ports)
router3_ifaces   = router3.get_component("nic1").get_interfaces()     # CX6 (2 ports)
router4_ifaces   = router4.get_component("nic1").get_interfaces()     # CX6 (2 ports)

In [ ]:
# Define which physical port does what:
router1_wan_base    = router1_ifaces[0]   # will carry VLAN sub-interfaces
router1_sender_port = router1_ifaces[1]   # flat LAN from senders (no VLAN)

router2_wan_base    = router2_ifaces[0]   # will carry VLAN sub-interfaces
router2_sender_port = router2_ifaces[1]   # flat LAN from senders (no VLAN)

router3_wan_base    = router3_ifaces[1]   # will carry VLAN sub-interfaces
router3_receiver_port = router3_ifaces[0]   # flat LAN from receivers (no VLAN)

router4_wan_base    = router4_ifaces[1]   # will carry VLAN sub-interfaces
router4_receiver_port = router4_ifaces[0]   # flat LAN from receivers (no VLAN)

In [ ]:
inter_router1_sender_port   = inter_router1.get_component("nic1").get_interfaces()[0]
inter_router1_router_port   = inter_router1.get_component("nic2").get_interfaces()[0]

inter_router2_sender_port   = inter_router2.get_component("nic1").get_interfaces()[0]
inter_router2_router_port   = inter_router2.get_component("nic2").get_interfaces()[0]

inter_router3_sender_port   = inter_router3.get_component("nic1").get_interfaces()[0]
inter_router3_router_port   = inter_router3.get_component("nic2").get_interfaces()[0]

inter_router4_sender_port   = inter_router4.get_component("nic1").get_interfaces()[0]
inter_router4_router_port   = inter_router4.get_component("nic2").get_interfaces()[0]

In [ ]:
router1_wan_base.get_device_name()

In [ ]:
router1_sender_port.get_device_name()

In [ ]:
router2_sender_port.get_device_name()

In [ ]:
router2_wan_base.get_device_name()

In [ ]:
router4_wan_base.get_device_name()

In [ ]:
router4_receiver_port.get_device_name()

In [ ]:
router3_receiver_port.get_device_name()

In [ ]:
router3_wan_base.get_device_name()

In [ ]:
#receiver_wan_base  = receiver_ifaces[0] # VLAN sub-interfaces counterpart

# 2) Build the sender LAN (no VLAN) exactly as in net_conf[0]
lan = next(n for n in net_conf if n["name"] == "net1-sender-intermediate-router-1")
lan_ifaces = [
    sender1.get_component("nic1").get_interfaces()[0],
    sender2.get_component("nic1").get_interfaces()[0],
    inter_router1_sender_port
]

net1_sender_intermediate_router_1 = slice.add_l2network(name=lan["name"], interfaces=lan_ifaces, type="L2Bridge")

lan2 = next(n for n in net_conf if n["name"] == "net1-sender-intermediate-router-2")
lan_ifaces = [
    sender3.get_component("nic1").get_interfaces()[0],
    sender4.get_component("nic1").get_interfaces()[0],
    inter_router2_sender_port
]

net1_sender_intermediate_router_2 = slice.add_l2network(name=lan2["name"], interfaces=lan_ifaces, type="L2Bridge")


lan3 = next(n for n in net_conf if n["name"] == "intermediate-router1")
lan_ifaces = [
    inter_router1_router_port,
    inter_router2_router_port,
    router1_sender_port

]

net_intermediate-router1 = slice.add_l2network(name=lan3["name"], interfaces=lan_ifaces, type="L2Bridge")


###

lan = next(n for n in net_conf if n["name"] == "net2-sender-intermediate-router-1")
lan_ifaces = [
    sender5.get_component("nic1").get_interfaces()[0],
    sender6.get_component("nic1").get_interfaces()[0],
    inter_router3_sender_port
]

net1_sender_intermediate_router_1 = slice.add_l2network(name=lan["name"], interfaces=lan_ifaces, type="L2Bridge")

lan2 = next(n for n in net_conf if n["name"] == "net2-sender-intermediate-router-2")
lan_ifaces = [
    sender7.get_component("nic1").get_interfaces()[0],
    sender8.get_component("nic1").get_interfaces()[0],
    inter_router4_sender_port
]

net1_sender_intermediate_router_2 = slice.add_l2network(name=lan2["name"], interfaces=lan_ifaces, type="L2Bridge")


lan3 = next(n for n in net_conf if n["name"] == "intermediate-router2")
lan_ifaces = [
    inter_router3_router_port,
    inter_router4_router_port,
    router2_sender_port

]

net_intermediate-router2 = slice.add_l2network(name=lan3["name"], interfaces=lan_ifaces, type="L2Bridge")

###


lan3 = next(n for n in net_conf if n["name"] == "net1-receiver")
lan3_ifaces = [
    receiver1.get_component("nic1").get_interfaces()[0],
    receiver2.get_component("nic1").get_interfaces()[0],
    receiver5.get_component("nic1").get_interfaces()[0],
    receiver6.get_component("nic1").get_interfaces()[0],
    router3_receiver_port
]

net1_receiver = slice.add_l2network(name=lan3["name"], interfaces=lan3_ifaces, type="L2Bridge")


lan4 = next(n for n in net_conf if n["name"] == "net2-receiver")
lan4_ifaces = [
    receiver3.get_component("nic1").get_interfaces()[0],
    receiver4.get_component("nic1").get_interfaces()[0],
    receiver7.get_component("nic1").get_interfaces()[0],
    receiver8.get_component("nic1").get_interfaces()[0],
    router4_receiver_port
]

net2_receiver = slice.add_l2network(name=lan4["name"], interfaces=lan4_ifaces, type="L2Bridge")


# 3) Build WAN tunnels using VLAN sub-interfaces (use your net_conf entries as-is)
wan_defs = [n for n in net_conf if "router1-router3" in n["name"]]
for wn in wan_defs:
    vlan   = str(wn["vlan"])
    hops   = [wn["hops"]]
    bw_g   = wn["bw"]                   # Gbps

    # Create matching VLAN sub-ifaces on router_wan_base and receiver_wan_base
    r_sub = router1_wan_base.add_sub_interface(name=f"r-{vlan}", vlan=vlan)
    x_sub = router3_wan_base.add_sub_interface(name=f"x-{vlan}", vlan=vlan)
    r_sub.set_mode('auto')
    x_sub.set_mode('auto')

    # L2PTP for this tunnel; use your subnet for addressing
    ns = slice.add_l2network(name=wn["name"], interfaces=[r_sub, x_sub], type="L2PTP")
    ns.set_l2_route_hops(hops=hops)
    ns.set_bandwidth(bw_g)

wan_defs = [n for n in net_conf if "router1-router4" in n["name"]]
for wn in wan_defs:
    vlan   = str(wn["vlan"])
    hops   = [wn["hops"]]
    bw_g   = wn["bw"]                   # Gbps

    # Create matching VLAN sub-ifaces on router_wan_base and receiver_wan_base
    r_sub = router1_wan_base.add_sub_interface(name=f"r-{vlan}", vlan=vlan)
    x_sub = router4_wan_base.add_sub_interface(name=f"x-{vlan}", vlan=vlan)
    r_sub.set_mode('auto')
    x_sub.set_mode('auto')

    # L2PTP for this tunnel; use your subnet for addressing
    ns = slice.add_l2network(name=wn["name"], interfaces=[r_sub, x_sub], type="L2PTP")
    ns.set_l2_route_hops(hops=hops)
    ns.set_bandwidth(bw_g)

wan_defs = [n for n in net_conf if "router2-router3" in n["name"]]
for wn in wan_defs:
    vlan   = str(wn["vlan"])
    hops   = [wn["hops"]]
    bw_g   = wn["bw"]                   # Gbps

    # Create matching VLAN sub-ifaces on router_wan_base and receiver_wan_base
    r_sub = router2_wan_base.add_sub_interface(name=f"r-{vlan}", vlan=vlan)
    x_sub = router3_wan_base.add_sub_interface(name=f"x-{vlan}", vlan=vlan)
    r_sub.set_mode('auto')
    x_sub.set_mode('auto')

    # L2PTP for this tunnel; use your subnet for addressing
    ns = slice.add_l2network(name=wn["name"], interfaces=[r_sub, x_sub], type="L2PTP")
    ns.set_l2_route_hops(hops=hops)
    ns.set_bandwidth(bw_g)

wan_defs = [n for n in net_conf if "router2-router4" in n["name"]]
for wn in wan_defs:
    vlan   = str(wn["vlan"])
    hops   = [wn["hops"]]
    bw_g   = wn["bw"]                   # Gbps

    # Create matching VLAN sub-ifaces on router_wan_base and receiver_wan_base
    r_sub = router2_wan_base.add_sub_interface(name=f"r-{vlan}", vlan=vlan)
    x_sub = router4_wan_base.add_sub_interface(name=f"x-{vlan}", vlan=vlan)
    r_sub.set_mode('auto')
    x_sub.set_mode('auto')

    # L2PTP for this tunnel; use your subnet for addressing
    ns = slice.add_l2network(name=wn["name"], interfaces=[r_sub, x_sub], type="L2PTP")
    ns.set_l2_route_hops(hops=hops)
    ns.set_bandwidth(bw_g)

In [ ]:
wan_defs

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

In [ ]:
for node in slice.get_nodes():
    print(f"{node.get_name()} == {node.get_host()}")

In [ ]:
groups = {
    "S1-S4 (NCSA)": [],
    "R1,R2,R5,R6 (KANS)": [],
    "S5-S8 (MICH)": [],
    "R3,R4,R7,R8 (INDI)": [],
}

for node in slice.get_nodes():
    name = node.get_name()
    host = node.get_host()

    if host.startswith("ncsa"):
        groups["S1-S4 (NCSA)"].append(f"{name} == {host}")

    elif host.startswith("kans"):
        groups["R1,R2,R5,R6 (KANS)"].append(f"{name} == {host}")

    elif host.startswith("mich"):
        groups["S5-S8 (MICH)"].append(f"{name} == {host}")

    elif host.startswith("indi"):
        groups["R3,R4,R7,R8 (INDI)"].append(f"{name} == {host}")


for group_name, items in groups.items():
    print(f"\n=== {group_name} ===")
    for entry in items:
        print(entry)


### Pin CPU & NUMA Tuning

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
    # Pin all vCPUs for VM to same Numa node as the component

    try:
        node.pin_cpu(component_name="nic1") # also apply for nic2 for inter-routers

    except Exception as e:
        print(f"Warning: could not pin cpu for {node.get_name()}: {e}")

    # User can also pass in the range of the vCPUs to be pinned
    #node.pin_cpu(component_name=nic_name, cpu_range_to_pin="0-3")

    # Pin memmory for VM to same Numa node as the components
    #node.numa_tune()
    try:
        node.numa_tune()

    except Exception as e:
        print(f"Warning: could not pin memory for {node.get_name()}: {e}")

    # Reboot the VM
    node.os_reboot()

In [ ]:
slice.wait_ssh(progress=True)

In [ ]:
kernel_nodes = [f"sender{i}" for i in range(1, 9)] + [f"receiver{i}" for i in range(1, 9)]

sender_nodes=[f"sender{i}" for i in range(1, 9)]
receiver_nodes=[f"receiver{i}" for i in range(1, 9)]

router_nodes= [f"router{i}" for i in range(1, 5)]

inter_router_nodes= [f"inter-router{i}" for i in range(1, 5)]

### Install BBR3 Kernel

In [ ]:
pkg_list = ['linux-headers-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-image-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-libc-dev_6.4.0-g6e321d1c986a-5_amd64.deb']

cmd_BBRv3 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 6.4.0"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

# for pkg in pkg_list:
#     slice.get_node(name="sender").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
#     slice.get_node(name="receiver").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))

# slice.get_node(name="sender").execute("sudo dpkg -i  *.deb")
# slice.get_node(name="sender").execute(cmd_BBRv3)

# slice.get_node(name="receiver").execute("sudo dpkg -i  *.deb")
# slice.get_node(name="receiver").execute(cmd_BBRv3)


for node_name in kernel_nodes:
    node = slice.get_node(name=node_name)

    # download all .deb files
    for pkg in pkg_list:
        node.execute(f"wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{pkg}")

    # install and switch default kernel to 6.4.0
    node.execute("sudo dpkg -i *.deb")
    node.execute(cmd_BBRv3)

In [ ]:
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

### IOMMU

In [ ]:
grub_script = r"""
set -e

# Replace GRUB_CMDLINE_LINUX line
sudo sed -i 's/^GRUB_CMDLINE_LINUX=.*/GRUB_CMDLINE_LINUX="amd_iommu=on iommu=pt"/' /etc/default/grub

# Update grub config
sudo update-grub

# Reboot to apply
sudo reboot
"""

for name in kernel_nodes:

    node1=slice.get_node(name)
    print(f"\n=== Updating GRUB on {node1.get_name()} ===")
    node1.execute(grub_script)

In [ ]:
slice.wait_ssh(progress=True)

In [ ]:
for name in kernel_nodes:
    node1=slice.get_node(name)
    print(f"\n=== Checking on {node1.get_name()} ===")
    node1.execute("cat /proc/cmdline")

### DPDK kernels

In [ ]:
for name in inter_router_nodes:
#for name in router_nodes:

    node1=slice.get_node(name)
    print(f"\n=== Updating kernel on {node1.get_name()} ===")
    node1.execute("sudo apt update")
    node1.execute("sudo apt install -y build-essential libnuma-dev python3-pip rdma-core ibverbs-providers")
    node1.execute("pip install gdown")
    node1.execute("~/.local/bin/gdown https://drive.google.com/uc?id=15tCjwuM0KmKajgoqApiXgfU30KuFwBgg")
    node1.execute("sudo dpkg -i kernel-deb.deb")
    node1.execute("sudo update-grub")
    node1.execute("sudo reboot")

In [ ]:
slice.wait_ssh(progress=True)

### Apply nohz_full.

In [ ]:
# apply these steps on the terminal on the DPDK routers

# sudo nano /boot/grub/grub.cfg
# linux   /boot/vmlinuz-6.5.0NO-HZ-FULL root=/dev/vda1 ro console=tty1 console=ttyS0 nohz_full=8-15 rcu_nocbs=8-15 isolcpus=domain,managed_irq,8-15 housekeeping=on,0-7 amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=128

# console=tty1 console=ttyS0 nohz_full=56-63 rcu_nocbs=56-63 isolcpus=domain,managed_irq,56-63 housekeeping=on,0-55 amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=128

# sudo reboot

#cat /proc/cmdline
#cat /sys/devices/system/cpu/nohz_full

# sudo grep -R "vmlinuz" /boot/grub* /boot/efi 2>/dev/null | head -20

### Download DPDK Application

In [ ]:
for name in inter_router_nodes:
#for name in router_nodes:
    node1=slice.get_node(name)
    print(f"\n=== Updating kernel on {node1.get_name()} ===")
    node1.execute("sudo apt install unzip")
    node1.execute("~/.local/bin/gdown https://drive.google.com/uc?id=1_qQFSqtYHslrcCKyUYuk49_leHfO1d1E")
    node1.execute("sudo unzip DPDK-RoutingAwareShaper-main.zip")

### DPDK libraries

In [ ]:
for name in inter_router_nodes:
#for name in router_nodes:
    node1=slice.get_node(name)
    print(f"\n=== Installing DPDK libraries {node1.get_name()} ===")
    node1.upload_directory('/home/fabric/work/DPDK/node_toolsv2','.')
    node1.execute('chmod +x node_toolsv2/install.sh')
    node1.execute('sudo ./node_toolsv2/install.sh')

    node1.execute('chmod +x node_toolsv2/grub.sh')
    node1.execute('chmod +x node_toolsv2/apply_vfio_settings.sh')

    stdout, stderr = node1.execute(f"sudo node_toolsv2/grub.sh {node1.get_ram()}")
    stdout, stderr = node1.execute("sudo node_toolsv2/apply_vfio_settings.sh")

    node1.execute("sudo apt-get update")
    node1.execute("sudo apt-get install -y libibverbs-dev rdma-core ibverbs-utils pkg-config")
    node1.execute("sudo apt-get install -y libelf-dev libssl-dev libbsd-dev libarchive-dev libfdt-dev libjansson-dev")

In [ ]:
for name in inter_router_nodes:
#for name in router_nodes:
    node1=slice.get_node(name)
    print(f"\n=== Hugepages -- {node1.get_name()} ===")

    node1.execute("echo 100 | sudo tee /sys/kernel/mm/hugepages/hugepages-1048576kB/nr_hugepages")
    node1.execute("sudo mkdir -p /mnt/huge")
    node1.execute("sudo mount -t hugetlbfs nodev /mnt/huge -o pagesize=1G,mode=01777")
    node1.execute("mount | grep hugetlbfs")

### DPDK/TM10 installation

In [ ]:
install_cmd = r'''
set -xe

mkdir -p $HOME/DPDK/v22.11.10
cd $HOME/DPDK/v22.11.10

# Download only if missing
if [ ! -f dpdk-22.11.10.tar.xz ]; then
    wget https://fast.dpdk.org/rel/dpdk-22.11.10.tar.xz
fi

# Extract only if missing
if [ ! -d dpdk-stable-22.11.10 ]; then
    tar xvJf dpdk-22.11.10.tar.xz
fi

cd dpdk-stable-22.11.10

meson --reconfigure --prefix=~/dpdk22.11Install -Dexamples=l2fwd,l3fwd,vhost build

cd build
ninja
'''
for name in inter_router_nodes:
#for name in router_nodes:
    node = slice.get_node(name)
    print(f"\n=== Installing DPDK v22.11.10 on {name} ===")

    print(node.execute(f"bash -c \"{install_cmd}\"")[0])


In [ ]:
cmd = r'''
set -xe

BASE=$HOME/DPDK/v22.11.10/dpdk-stable-22.11.10
SRC=$HOME/DPDK-RoutingAwareShaper-main/DaaS

# Make sure base exists
mkdir -p "$BASE"

# Remove any old copy to avoid weird leftovers
rm -rf "$BASE/DaaS"

# Copy fresh DaaS folder
cp -r "$SRC" "$BASE/"

# Show result
ls -ld "$BASE/DaaS"
ls "$BASE/DaaS"
'''

cmd2 = r'''
set -xe

BASE=$HOME/DPDK/v22.11.10/dpdk-stable-22.11.10

cd "$BASE"

# 1. First-time build (runs only if build/ does not exist)
if [ ! -d build ]; then
    echo ">>> Running initial meson..."
    meson --prefix=$HOME/dpdk22.11Install -Dexamples=l2fwd,l3fwd,vhost -Dopocp2=all build
fi

# 2. Reconfigure (always safe)
echo ">>> Reconfiguring meson..."
meson --reconfigure --prefix=$HOME/dpdk22.11Install -Dexamples=l2fwd,l3fwd,vhost build

# 3. Build tm10 and others
echo ">>> Running ninja build..."
cd build
ninja
'''

for name in inter_router_nodes:
#for name in router_nodes:
    node = slice.get_node(name)
    print(f"\n=== Copying DaaS into DPDK tree on {name} ===")

    out = node.execute(f"bash -c '{cmd}'")
    print(out[0])

    node.upload_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/meson_options.txt",f"/home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/meson_options.txt")
    node.upload_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/meson.build",f"/home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/meson.build")

    output = node.execute(f"bash -c '{cmd2}'")[0]
    print(output)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice=fablib.get_slice(slice_name)

slice.wait_ssh()

for i in slice.get_interfaces():
    i.config_vlan_iface()
    i.config()

for n in slice.get_nodes():
    n.config()
    #n.execute('sudo node_tools/host_tune_enable_docker.sh', quiet=True, output_file=f"{n.get_name()}.log")

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
from ipaddress import IPv4Network

all_ifaces = slice.get_interfaces()

for net in net_conf:
    net_name = net["name"]
    subnet   = IPv4Network(net["subnet"])

    for n in net["nodes"]:
        node_name = n["name"]
        addr      = n["addr"]

          # --- Sender LANs (no VLAN, p1 on senders, p2 on router1/2) ---
        if net_name in ("net1-sender-intermediate-router-1", "net1-sender-intermediate-router-2","net2-sender-intermediate-router-1","net2-sender-intermediate-router-2"):
             if_name = f"{node_name}-nic1-p1"


        # --- Receiver LANs (no VLAN, p1 on receivers, p2 on router3/4) ---
        elif net_name in ("net1-receiver", "net2-receiver"):
            if node_name.startswith("router"):

                if node_name=="router4":
                    if_name = f"{node_name}-nic1-p1"   #router4
                if node_name=="router3":
                    if_name = f"{node_name}-nic1-p2"   #router3
            else:
                if_name = f"{node_name}-nic1-p1"   # receiver1..8

        elif net_name in ("intermediate-router1", "intermediate-router2"):
            if node_name.startswith("inter"):
                if_name = f"{node_name}-nic2-p1"   # receiver1..8

            else:
                if_name = f"{node_name}-nic1-p2"   # receiver1..8

        # --- WAN tunnels: VLAN sub-interfaces on p1 ---
        else:
            vlan = net["vlan"]

            if "router1-router3" in net_name:
                if node_name == "router1":
                    if_name = f"{node_name}-nic1-p1-r-{vlan}"
                elif node_name == "router3":
                    if_name = f"{node_name}-nic1-p1-x-{vlan}"
                else:
                    print(f"[skip] {node_name} not on {net_name}")
                    continue

            elif "router1-router4" in net_name:
                if node_name == "router1":
                    if_name = f"{node_name}-nic1-p1-r-{vlan}"
                elif node_name == "router4":
                    if_name = f"{node_name}-nic1-p2-x-{vlan}"
                else:
                    print(f"[skip] {node_name} not on {net_name}")
                    continue

            elif "router2-router3" in net_name:
                if node_name == "router2":
                    if_name = f"{node_name}-nic1-p1-r-{vlan}"
                elif node_name == "router3":
                    if_name = f"{node_name}-nic1-p1-x-{vlan}"
                else:
                    print(f"[skip] {node_name} not on {net_name}")
                    continue

            elif "router2-router4" in net_name:
                if node_name == "router2":
                    if_name = f"{node_name}-nic1-p1-r-{vlan}"
                elif node_name == "router4":
                    if_name = f"{node_name}-nic1-p2-x-{vlan}"
                else:
                    print(f"[skip] {node_name} not on {net_name}")
                    continue
            else:
                print(f"[skip] Unknown net {net_name}")
                continue

        iface_list = [i for i in all_ifaces if i.get_name() == if_name]

        if not iface_list:
            print(f"[skip] interface {if_name} not found (exact match)")
            continue

        iface = iface_list[0]

        iface.ip_link_up()
        iface.ip_addr_add(addr=addr, subnet=subnet)
        print(f"{net_name}: {node_name} -> {if_name} ({addr}/{subnet.prefixlen})")

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
#set up static routes
for rt in route_conf:
   for n in rt['nodes']:
       slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# # turn off segmentation offload on interfaces
# for iface in slice.get_interfaces():
#     iface_name = iface.get_device_name()
#     n = iface.get_node()
#     offloads = ["gro", "lro", "gso", "tso"]
#     for offload in offloads:
#         n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Get Interfaces

In [ ]:
for node in slice.get_nodes():
    print(f"\n=== {node.get_name()} ===")
    output = node.execute("ls /sys/class/net")[0]
    print(output)

In [ ]:
node_interfaces = {}

for node in slice.get_nodes():
    name = node.get_name()

    # get all interfaces
    raw = node.execute("ls /sys/class/net")[0].strip().split()

    # filter: remove lo + enp3s0 (mgmt)
    ifaces = [
        iface for iface in raw
        if iface != "lo" and iface != "enp3s0"
    ]

    node_interfaces[name] = ifaces

In [ ]:
node_interfaces

### IRQ AFFINITY

In [ ]:
# Step 1: Stop irqbalance + download scripts
prepare_script = r"""
set -e

# Stop and disable irqbalance
sudo systemctl stop irqbalance || true
sudo systemctl disable irqbalance || true

# Download Mellanox IRQ affinity scripts
wget -q https://raw.githubusercontent.com/egobillot/mlnx-irq-tools/master/common_irq_affinity.sh -O common_irq_affinity.sh
wget -q https://raw.githubusercontent.com/Mellanox/mlnx-tools/mlnx_ofed/sbin/set_irq_affinity_bynode.sh -O set_irq_affinity_bynode.sh
wget -q https://raw.githubusercontent.com/egobillot/mlnx-irq-tools/master/show_irq_affinity.sh -O show_irq_affinity.sh

# Make executable
chmod +x common_irq_affinity.sh
chmod +x set_irq_affinity_bynode.sh
chmod +x show_irq_affinity.sh
"""

# Prepare each endpoint
for name in kernel_nodes:
    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    print(f"\n=== Preparing {name} ===")
    node = slice.get_node(name)
    node.execute(prepare_script)

# Step 2: Apply IRQ affinity
for name in kernel_nodes:
    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Applying IRQ affinity on {name} ===")
    print(f"Interfaces: {ifaces}")

    node = slice.get_node(name)

    for iface in ifaces:
        cmd = f"sudo ./set_irq_affinity_bynode.sh 0 {iface}"
        print(f"  -> {cmd}")
        node.execute(cmd)

In [ ]:
# Step 3: Verify IRQ affinity
for name in kernel_nodes:
#for name in routers_check:
    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Checking IRQ affinity on {name} ===")
    print(f"Interfaces: {ifaces}")

    node = slice.get_node(name)

    for iface in ifaces:
        cmd = f"sudo ./show_irq_affinity.sh {iface}"
        print(f"  -> {cmd}")
        node.execute(cmd)

In [ ]:
cmd = r'''
set -xe

mkdir -p ~/DPDK/utils
cd ~/DPDK/utils

# Download scripts if missing
test -f set_irq_affinity_cpulist.sh || wget -q https://raw.githubusercontent.com/egobillot/mlnx-irq-tools/master/set_irq_affinity_cpulist.sh -O set_irq_affinity_cpulist.sh
test -f show_irq_affinity.sh        || wget -q https://raw.githubusercontent.com/egobillot/mlnx-irq-tools/master/show_irq_affinity.sh -O show_irq_affinity.sh
test -f common_irq_affinity.sh      || wget -q https://raw.githubusercontent.com/egobillot/mlnx-irq-tools/master/common_irq_affinity.sh -O common_irq_affinity.sh

chmod +x *.sh
'''

for name in inter_router_nodes:
    node = slice.get_node(name)
    print(f"\n=== Preparing utils/ on {name} ===")


    out = node.execute(f"bash -c '{cmd}'")[0]
    print(out)


In [ ]:
# -------------------------------
# Step 2: Apply IRQ Affinity
# -------------------------------
for name in inter_router_nodes:
    if name not in node_interfaces:
        print(f"{name}: no interface info → skipping")
        continue

    if name=='inter-router4':

        for iface in ifaces:
            cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/set_irq_affinity_cpulist.sh 0-1 {iface}"
            print(f"  -> {cmd}")
            print(node.execute(cmd)[0])

        print("skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Applying IRQ affinity on {name} ===")
    print(f"Interfaces: {ifaces}")

    node = slice.get_node(name)

    for iface in ifaces:
        cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/set_irq_affinity_cpulist.sh 0-7 {iface}"
        #print(f"  -> {cmd}")
        print(node.execute(cmd)[0])

In [ ]:
# -------------------------------
# Step 2: Apply IRQ Affinity
# -------------------------------
#for name in inter_router_nodes:
for name in router_nodes:
    if name not in node_interfaces:
        print(f"{name}: no interface info → skipping")
        continue

    if name=='inter-router4':

        for iface in ifaces:
            cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/set_irq_affinity_cpulist.sh 0-1 {iface}"
            print(f"  -> {cmd}")
            print(node.execute(cmd)[0])

        print("skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Applying IRQ affinity on {name} ===")
    print(f"Interfaces: {ifaces}")

    node = slice.get_node(name)

    for iface in ifaces:
        #cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/set_irq_affinity_cpulist.sh 0-7 {iface}"
        cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/set_irq_affinity_cpulist.sh 0-55 {iface}"

        #print(f"  -> {cmd}")
        print(node.execute(cmd)[0])

In [ ]:
# -------------------------------
# Step 3: Verify IRQ Affinity
# -------------------------------
for name in router_nodes:
#for name in inter_router_nodes:
    if name not in node_interfaces:
        print(f"{name}: no interface info → skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Checking IRQ affinity on {name} ===")
    print(f"Interfaces: {ifaces}")

    node = slice.get_node(name)

    for iface in ifaces:
        cmd = f"cd ~/DPDK/utils && sudo ~/DPDK/utils/show_irq_affinity.sh {iface}"
        print(f"  -> {cmd}")
        print(node.execute(cmd)[0])

### Validate base network

Before we run any experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
receiver1_addr="10.0.11.100"
receiver2_addr="10.0.11.110"

receiver3_addr="10.0.12.140"
receiver4_addr="10.0.12.150"


receiver5_addr="10.0.11.120"
receiver6_addr="10.0.11.130"

receiver7_addr="10.0.12.120"
receiver8_addr="10.0.12.130"


sender1_addr="10.0.1.140"
sender2_addr="10.0.1.150"

sender3_addr="10.0.14.120"
sender4_addr="10.0.14.130"

sender5_addr="10.0.2.140"
sender6_addr="10.0.2.150"

sender7_addr="10.0.15.120"
sender8_addr="10.0.15.130"

### New Configuration

In [ ]:
sender1   = slice.get_node("sender1")
sender2  = slice.get_node("sender2")
sender3  = slice.get_node("sender3")
sender4  = slice.get_node("sender4")

inter_router1   = slice.get_node("inter-router1")
inter_router2  = slice.get_node("inter-router2")


sender5   = slice.get_node("sender5")
sender6  = slice.get_node("sender6")
sender7  = slice.get_node("sender7")
sender8  = slice.get_node("sender8")

inter_router3   = slice.get_node("inter-router3")
inter_router4  = slice.get_node("inter-router4")


router1   = slice.get_node("router1")
router2  = slice.get_node("router2")
router3   = slice.get_node("router3")
router4   = slice.get_node("router4")


receiver1 = slice.get_node("receiver1")
receiver2 = slice.get_node("receiver2")
receiver3 = slice.get_node("receiver3")
receiver4 = slice.get_node("receiver4")

receiver5 = slice.get_node("receiver5")
receiver6 = slice.get_node("receiver6")
receiver7 = slice.get_node("receiver7")
receiver8 = slice.get_node("receiver8")

In [ ]:
senders=[sender1,sender2,sender3,sender4,sender5,sender6,sender7,sender8]
receivers=[receiver1,receiver2,receiver3,receiver4,receiver5,receiver6,receiver7,receiver8]

### PING TEST

In [ ]:
for i in range(1, 9):
    #sender = slice.get_node(f"sender{i}")
    sender =  senders[i-1]
    dest_ip = globals()[f"receiver{i}_addr"]   # "receiver1_addr" → receiver1_addr
    print(f"\n=== sender{i} → {dest_ip} ===")
    sender.execute(f"ping -c 5 {dest_ip}")

### Throughput TEST

In [ ]:
for i in range(1, 9):
    #sender = slice.get_node(f"sender{i}")
    #receiver = slice.get_node(f"receiver{i}")
    sender =  senders[i-1]
    receiver = receivers[i-1]


    dest_ip = globals()[f"receiver{i}_addr"]   # "receiver1_addr" → receiver1_addr
    sender.execute("sudo modprobe tcp_bbr1")
    receiver.execute("sudo modprobe tcp_bbr1")
    print(f"\n=== sender{i} → {dest_ip} ===")
    receiver.execute("iperf3 -s -1 -D")
    time.sleep(5)
    sender.execute(f"iperf3 -t 30 -P 10 -i 10 -C bbr1 -c {dest_ip}")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'],
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
filtered = [
    entry
    for entry in ifaces
    if entry.get("ip_addr") not in (None, "None")
]

In [ ]:
ifaces_new=filtered

In [ ]:
clean_edges = [
    (u, v, attr)
    for (u, v, attr) in edges
    if (u != 'None' and v != 'None')
]

In [ ]:
edges_new=clean_edges

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges_new)

plt.figure(figsize=(24, 18), dpi=200)

pos = nx.spring_layout(
    G,
    k=4.0,          # try 2.0, 4.0, 8.0
    iterations=1000,
    seed=1
)

nx.draw_networkx_edges(G, pos, alpha=0.35, width=1.2)
nx.draw_networkx_nodes(
    G, pos,
    node_shape='s',
    node_color=[n[1]['color'] for n in nodes],
    node_size=[len(n[0]) * 350 for n in nodes],
    edgecolors='black',
    linewidths=0.8,
)
nx.draw_networkx_labels(G, pos, font_size=8,
                        bbox=dict(facecolor='white', edgecolor='none', alpha=0.6))

# Edge labels are usually what makes it unreadable; consider turning off first
# nx.draw_networkx_edge_labels(...)

plt.axis("off")
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges_new)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',
        node_color=[n[1]['color'] for n in nodes],
        node_size=[len(n[0])*400 for n in nodes],
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

In [ ]:
senders_check=["sender1","sender2","sender3","sender4","sender5","sender6","sender7","sender8"]
receivers_check=["receiver1","receiver2","receiver3","receiver4","receiver5","receiver6","receiver7","receiver8"]
routers_check=["router1","router2","router3","router4"]
inter_router_nodes_check=['inter-router1', 'inter-router2', 'inter-router3', 'inter-router4']

### Tune Hosts

In [ ]:
config = r"""sudo tee /etc/sysctl.d/99-hpwan-tuned.conf >/dev/null <<'EOF'
# --- payload/socket buffer limits (bytes) ---
net.core.wmem_max = 2147483647
net.core.rmem_max = 2147483647

# tcp mem are in PAGES (three values: low/pressure/high)
net.ipv4.tcp_mem = 2147483647 2147483647 2147483647

# tcp per-socket autotune (min/default/max in BYTES)
net.ipv4.tcp_rmem = 4096 131072 2147483647
net.ipv4.tcp_wmem = 4096 16384 2147483647

# backlog for ingress (NIC -> kernel queue)
net.core.netdev_max_backlog = 250000

# probing + metrics behavior
net.ipv4.tcp_mtu_probing = 1
net.ipv4.tcp_no_metrics_save = 1

# timestamps on
net.ipv4.tcp_timestamps = 1

# default qdisc
net.core.default_qdisc = fq

# enable MSG_ZEROCOPY
net.core.optmem_max = 1048576
EOF

sudo sysctl --system
"""

# all endpoints, no routers
kernel_nodes = [f"sender{i}" for i in range(1, 9)] + [f"receiver{i}" for i in range(1, 9)]

#for name in kernel_nodes:
for name in routers_check:

    node1=slice.get_node(name)

    print(f"Configuring {node1.get_name()}...")

    node1.execute(config)

In [ ]:
#for name in kernel_nodes:
for name in senders_check:

    node1=slice.get_node(name)

    print(node1.execute("sysctl net.core.wmem_max"))
    print(node1.execute("sysctl net.ipv4.tcp_wmem"))
    print(node1.execute("sysctl net.core.default_qdisc"))

### iperf3 update

In [ ]:
install_script = r"""
set -e

# Update packages and install dependencies
sudo apt update -y
sudo apt install -y build-essential gcc make libssl-dev libsctp-dev curl

# Remove old iperf3 versions
sudo rm -f /usr/local/bin/iperf3
sudo rm -f /usr/local/lib/libiperf*
sudo rm -f /usr/local/include/iperf_api.h
sudo ldconfig

# Build iperf3 v3.19.1
cd /tmp
rm -rf iperf-3.19.1*
curl -LO https://github.com/esnet/iperf/releases/download/3.19.1/iperf-3.19.1.tar.gz
tar -xf iperf-3.19.1.tar.gz
cd iperf-3.19.1

./configure
make -j"$(nproc)"
sudo make install
sudo ldconfig

hash -r

# Verification
echo "Which iperf3:"
which iperf3 || true

echo "iperf3 version:"
iperf3 --version || true
"""

for node in slice.get_nodes():
    print(f"\n=== Installing iperf3 on {node.get_name()} ===")
    node.execute(install_script)

In [ ]:
for node in slice.get_nodes():
    print(f"\n=== Installing iperf3 on {node.get_name()} ===")
    node.execute("iperf3 --version")

In [ ]:

#for name in kernel_nodes:
for name in receivers_check:
    #name = node.get_name()

    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Configuring {name} ===")
    print(f"Interfaces: {ifaces}")


    node1=slice.get_node(name)

    for iface in ifaces:

        # 1. Increase txqueuelen
        cmd1 = f"sudo ip link set dev {iface} txqueuelen 10000"
        print(f"  -> {cmd1}")
        node1.execute(cmd1)

        # 2. Set jumbo MTU
        cmd2 = f"sudo ip link set dev {iface} mtu 9000"
        print(f"  -> {cmd2}")
        node1.execute(cmd2)

        # 3. Increase RX/TX ring sizes
        cmd3 = f"sudo ethtool -G {iface} rx 8192 tx 8192"
        print(f"  -> {cmd3}")
        node1.execute(cmd3)

        # 4. Enable tso / gso / gro
        cmd4 = f"sudo ethtool -K {iface} tso on gso on gro on"
        print(f"  -> {cmd4}")
        node1.execute(cmd4)

        # 5. Install fq qdisc with higher queue size
        cmd5 = f"sudo tc qdisc replace dev {iface} root handle 1: fq limit 20480 flow_limit 10240"
        print(f"  -> {cmd5}")
        node1.execute(cmd5)

In [ ]:

for name in inter_router_nodes_check:
    #name = node.get_name()

    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    ifaces = node_interfaces[name]
    print(f"\n=== Configuring {name} ===")
    print(f"Interfaces: {ifaces}")


    node1=slice.get_node(name)

    for iface in ifaces:

        # 1. Increase txqueuelen
        cmd1 = f"sudo ip link set dev {iface} txqueuelen 10000"
        print(f"  -> {cmd1}")
        node1.execute(cmd1)

        # 2. Set jumbo MTU
        cmd2 = f"sudo ip link set dev {iface} mtu 9000"
        print(f"  -> {cmd2}")
        node1.execute(cmd2)

        # 3. Increase RX/TX ring sizes
        cmd3 = f"sudo ethtool -G {iface} rx 8192 tx 8192"
        print(f"  -> {cmd3}")
        node1.execute(cmd3)

        # # 4. Replace root qdisc with stable handle 9001:
        # cmd4 = f"sudo tc qdisc replace dev {iface} root handle 9001: mq"
        # print(f"  -> {cmd4}")
        # node.execute(cmd4)


        # # 5. Find number of queues (using ethtool)
        # out, err = node.execute(f"ethtool -l {iface}", quiet=True)
        # queues = 1
        # for line in out.splitlines():
        #     if "Combined:" in line:
        #         try:
        #             queues = int(line.split(":")[1].strip())
        #         except:
        #             queues = 1
        # print(f"  -> Detected {queues} hardware queues")



        # 4. Enable tso / gso / gro
        #cmd4 = f"sudo ethtool -K {iface} tso on gso on gro on"
        #print(f"  -> {cmd4}")
        #node1.execute(cmd4)

        # 5. Install fq qdisc with higher queue size
        #cmd5 = f"sudo tc qdisc replace dev {iface} root handle 1: fq limit 20480 flow_limit 10240"
        #print(f"  -> {cmd5}")
        #node1.execute(cmd5)

### Permanent ARP

In [ ]:
import re

for name in kernel_nodes:

    if name not in node_interfaces:
        print(f"{name}: no interface info, skipping")
        continue

    node = slice.get_node(name)
    ifaces = node_interfaces[name]

    print(f"\n=== Setting permanent ARP entries on {name} ===")
    print(f"Interfaces: {ifaces}")

    for iface in ifaces:

        # Read neighbor table for this interface
        neigh_output = node.execute(f"ip neigh show dev {iface}")[0].strip().splitlines()

        for line in neigh_output:
            # Match pattern: "10.0.1.101 lladdr b8:ce:f6:37:75:1b STALE"
            m = re.match(r"^(\d+\.\d+\.\d+\.\d+)\s+lladdr\s+([0-9a-f:]+)", line)
            if not m:
                continue

            ip = m.group(1)
            mac = m.group(2)

            print(f"  -> {name}: {iface}: {ip} → {mac} (PERMANENT)")

            # Remove old entry
            node.execute(f"sudo ip neigh del {ip} dev {iface} || true")

            # Add permanent entry
            node.execute(f"sudo ip neigh add {ip} lladdr {mac} nud permanent dev {iface}")


In [ ]:
#for node in slice.get_nodes():
#    node.upload_directory('/home/fabric/work/Internship-work/node_tools','.')
#    node.execute('chmod +x node_tools/host_tune.sh')
#    node.execute('sudo sysctl net.ipv4.tcp_available_congestion_control')
#    node.execute('sudo ./node_tools/host_tune.sh')

### Experiments

In [ ]:
def build_drop_check_command(expname: str) -> str:
    """
    Build a self-contained bash command that:
      - auto-discovers interfaces (physical + VLANs on them)
      - checks ip-link, ethtool (real drop/error counters), tc qdisc/class
      - checks per-CPU softnet backlog drops and (if available) nstat TCP/IP counters
      - writes and tees output to /home/ubuntu/drop_check_<expname>.log
      - exits 0 if clean, 1 if any drops/errors found
    """
    prefix = f'EXP="{expname}"; '
    body = r'''
LOGFILE="/home/ubuntu/drop_check_${EXP}.log"
exec > >(tee -a "$LOGFILE") 2>&1

echo "=== Drop Check Started at $(date) ==="

# --- Auto-discover interfaces (physical + VLANs on them) ---
mapfile -t PHYS_IFACES < <(for d in /sys/class/net/*; do
  dev=$(basename "$d")
  [[ "$dev" == "lo" ]] && continue
  [[ -e "$d/device" ]] || continue   # only devices backed by hardware
  echo "$dev"
done)

mapfile -t VLAN_IFACES < <(ip -d -o link show type vlan 2>/dev/null | awk -F': ' '{print $2}' | while read -r pair; do
  ifname="${pair%%@*}"; lower="${pair##*@}"
  for p in "${PHYS_IFACES[@]}"; do
    if [[ "$lower" == "$p" ]]; then echo "$ifname"; break; fi
  done
done)

declare -A seen
IFACES=()
for x in "${PHYS_IFACES[@]}" "${VLAN_IFACES[@]}"; do
  [[ -n "$x" && -d "/sys/class/net/$x" && -z "${seen[$x]}" ]] && IFACES+=("$x") && seen[$x]=1
done

echo "Discovered interfaces: ${IFACES[*]}"
ok=true

# --- Helpers ---

check_ip_link() {
  ip -s link show dev "$1" 2>/dev/null | awk '
    /RX:/ { getline; split($0,a); rx_err=a[3]+0; rx_drop=a[4]+0 }
    /TX:/ { getline; split($0,b); tx_err=b[3]+0; tx_drop=b[4]+0 }
    END {
      if (rx_err || rx_drop || tx_err || tx_drop)
        printf "ip_link: RXerr=%d RXdrop=%d TXerr=%d TXdrop=%d\n", rx_err, rx_drop, tx_err, tx_drop
    }'
}

check_ethtool() {
  local IFACE="$1"
  local BASE="${IFACE%%.*}"
  (sudo ethtool -S "$IFACE" 2>/dev/null || sudo ethtool -S "$BASE" 2>/dev/null) | \
  awk -F':' '
    {
      k=$1; v=$2; gsub(/^[ \t]+|[ \t]+$/, "", v); lk=tolower(k)
      # Print only if >0; focus on true loss/error indicators.
      if (lk ~ /(rx_out_of_buffer|rx_dropped|tx_dropped|rx_errors|tx_errors|no_buffer|overflow|rx_missed_errors|discard|no_desc|fifo_errors|crc_errors|length_errors|rx_discards_phy|rx_prio[0-9]+_discards)/) {
        if ((v+0) > 0) printf "ethtool: %s %s\n", k, v+0
      }
    }'
}

check_tc_qdisc_class() {
  local IFACE="$1"
  sudo tc -s qdisc show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "qdisc: " $0 }'
  sudo tc -s class show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "class: " $0 }'
}

check_softnet_stat() {
  # Snapshot (no delta): column 2 per line (hex) = softnet backlog drops on that CPU.
  i=0
  while read -r line; do
    i=$((i+1)); set -- $line
    drops=$((16#$2))
    if (( drops > 0 )); then
      echo "softnet: CPU$((i-1)) backlog_drops(total) = $drops"
    fi
  done < /proc/net/softnet_stat
}

check_nstat() {
  # Optional: shows TCP/IP stack counters if iproute2's nstat is installed.
  if ! command -v nstat >/dev/null 2>&1; then
    return 0
  fi
  nstat -az 2>/dev/null | awk '
    $1 ~ /^(IpInDiscards|IpInErrors|TcpInErrs|TcpRetransSegs)$/ && $2>0 {
      printf "nstat: %s = %s\n", $1, $2
    }'
}

# --- Run per-interface checks ---
for IFACE in "${IFACES[@]}"; do
  echo "=== Checking $IFACE ==="
  DROPS=$(
    check_ip_link "$IFACE"
    check_ethtool "$IFACE"
    check_tc_qdisc_class "$IFACE"
  )
  if [[ -z "$DROPS" ]]; then
    echo "✅ No drops on $IFACE"
  else
    ok=false
    echo "❌ Drops/errors detected on $IFACE:"
    echo "$DROPS"
  fi
  echo
done

# --- System-wide checks ---
echo "=== Checking softnet backlog drops (per-CPU) ==="
SN=$(check_softnet_stat)
if [[ -z "$SN" ]]; then
  echo "✅ No softnet backlog drops"
else
  ok=false
  echo "❌ Softnet drops detected:"
  echo "$SN"
fi
echo

echo "=== Checking nstat (TCP/IP stack counters) ==="
NS=$(check_nstat)
if [[ -z "$NS" ]]; then
  echo "✅ nstat clean or not installed"
else
  ok=false
  echo "❌ Stack-level counters nonzero:"
  echo "$NS"
fi
echo

# --- Result ---
if $ok; then
  echo "✅✅ All interfaces clean — No drops anywhere."
  rc=0
else
  echo "⚠️ Some drops/errors detected above."
  rc=1
fi

echo "=== Drop Check Finished at $(date) ==="
echo "Results saved to $LOGFILE"
exit $rc
'''
    return prefix + body


def run_drop_check(node, expname: str):
    """Execute the drop-check on a given node with a chosen experiment name."""
    cmd = build_drop_check_command(expname)
    stdout, stderr = node.execute(cmd, quiet=True)


In [ ]:
import re

METRICS = [
    "rx_out_of_buffer",
    "rx_discards_phy",
    "rx_prio_discards",
    "ip_rxdrop",
    "ip_txdrop",
    "tc_qdisc_drop",
    "tc_class_drop",
    "retrans",
]


def parse_drop_log(log_text: str) -> dict:
    """
    Parse a single node's drop_check log into host-level counters.

    - Tracks per-interface stats under the current "=== Checking IFACE ===".
    - Groups interfaces by base name (e.g., enp7s0np0 and enp7s0np0.100 → base 'enp7s0np0').
    - For each base, takes the *max* across its interfaces (to avoid double-counting VLAN mirrors).
    - Sums across base interfaces to get host-level totals.
    - 'retrans' (TcpRetransSegs) is node-wide, not per-interface.
    """
    iface_counters = {}  # iface -> metric -> value
    host_retrans = 0
    current_iface = None

    def ensure_iface(iface):
        if iface not in iface_counters:
            iface_counters[iface] = {
                "rx_out_of_buffer": 0,
                "rx_discards_phy": 0,
                "rx_prio_discards": 0,
                "ip_rxdrop": 0,
                "ip_txdrop": 0,
                "tc_qdisc_drop": 0,
                "tc_class_drop": 0,
            }

    for raw in log_text.splitlines():
        line = raw.strip()

        # Detect interface section
        m = re.match(r"=== Checking (\S+) ===", line)
        if m:
            current_iface = m.group(1)
            ensure_iface(current_iface)
            continue

        # nstat (host-wide)
        m = re.match(r"nstat:\s+TcpRetransSegs\s+=\s+(\d+)", line)
        if m:
            host_retrans = int(m.group(1))
            continue

        if not current_iface:
            continue  # ignore lines outside an interface section

        # ip -s link RX/TX drops
        m = re.match(r"ip_link: RXerr=\d+ RXdrop=(\d+) TXerr=\d+ TXdrop=(\d+)", line)
        if m:
            rx_drop = int(m.group(1))
            tx_drop = int(m.group(2))
            ensure_iface(current_iface)
            iface_counters[current_iface]["ip_rxdrop"] += rx_drop
            iface_counters[current_iface]["ip_txdrop"] += tx_drop
            continue

        # ethtool stats
        m = re.match(r"ethtool:\s+(\S+)\s+(\d+)", line)
        if m:
            key = m.group(1)
            val = int(m.group(2))
            ensure_iface(current_iface)
            if key == "rx_out_of_buffer":
                # absolute counter
                iface_counters[current_iface]["rx_out_of_buffer"] = val
            elif key == "rx_discards_phy":
                iface_counters[current_iface]["rx_discards_phy"] = val
            elif re.match(r"rx_prio\d+_discards", key):
                # could be multiple priorities; sum them within the iface
                iface_counters[current_iface]["rx_prio_discards"] += val
            continue

        # tc qdisc drops
        m = re.search(r"qdisc: .*dropped (\d+)", line)
        if m:
            ensure_iface(current_iface)
            iface_counters[current_iface]["tc_qdisc_drop"] += int(m.group(1))
            continue

        # tc class drops
        m = re.search(r"class: .*dropped (\d+)", line)
        if m:
            ensure_iface(current_iface)
            iface_counters[current_iface]["tc_class_drop"] += int(m.group(1))
            continue

    # --- Group by base interface (e.g., enp7s0np0 vs enp7s0np0.100) ---
    grouped = {}  # base_iface -> metric -> value (max across variants)
    for iface, stats in iface_counters.items():
        base = iface.split(".")[0]
        if base not in grouped:
            grouped[base] = {k: 0 for k in stats.keys()}
        for k, v in stats.items():
            if v > grouped[base][k]:
                grouped[base][k] = v

    # --- Sum across base interfaces to get host-level totals ---
    total = {k: 0 for k in METRICS}
    for base, stats in grouped.items():
        for k in stats:
            total[k] += stats[k]

    total["retrans"] = host_retrans
    return total


def print_counter_diff(label: str, before: dict, after: dict):
    print(f"\n=== DIFF for {label} ===")
    for k in METRICS:
        b = before.get(k, 0)
        a = after.get(k, 0)
        diff = a - b
        print(f"{k:18s}: before={b:8d}  after={a:8d}  diff={diff:+d}")
    print("====================================\n")


In [ ]:
def build_drop_check_command(expname: str) -> str:
    """
    Updated:
      - ALWAYS prints counters (including zeros) so before/after deltas work reliably.
      - Adds /proc/net/netstat (TcpExt/IpExt) key counters that often explain sender-side retrans.
      - Adds ss -s summary (host-wide) to correlate with retrans/timewait/estab growth.
    """
    prefix = f'EXP="{expname}"; '
    body = r'''
LOGFILE="/home/ubuntu/drop_check_${EXP}.log"
exec > >(tee -a "$LOGFILE") 2>&1

echo "=== Drop Check Started at $(date) ==="
echo "HOST=$(hostname)"
echo

# --- Auto-discover interfaces (physical + VLANs on them) ---
mapfile -t PHYS_IFACES < <(for d in /sys/class/net/*; do
  dev=$(basename "$d")
  [[ "$dev" == "lo" ]] && continue
  [[ -e "$d/device" ]] || continue   # only devices backed by hardware
  echo "$dev"
done)

mapfile -t VLAN_IFACES < <(ip -d -o link show type vlan 2>/dev/null | awk -F': ' '{print $2}' | while read -r pair; do
  ifname="${pair%%@*}"; lower="${pair##*@}"
  for p in "${PHYS_IFACES[@]}"; do
    if [[ "$lower" == "$p" ]]; then echo "$ifname"; break; fi
  done
done)

declare -A seen
IFACES=()
for x in "${PHYS_IFACES[@]}" "${VLAN_IFACES[@]}"; do
  [[ -n "$x" && -d "/sys/class/net/$x" && -z "${seen[$x]}" ]] && IFACES+=("$x") && seen[$x]=1
done

echo "Discovered interfaces: ${IFACES[*]}"
ok=true

# --- Helpers ---

check_ip_link() {
  # ALWAYS print a single normalized line (even if all zeros)
  ip -s link show dev "$1" 2>/dev/null | awk '
    /RX:/ { getline; split($0,a); rx_err=a[3]+0; rx_drop=a[4]+0; rx_over=a[5]+0 }
    /TX:/ { getline; split($0,b); tx_err=b[3]+0; tx_drop=b[4]+0; tx_car=b[5]+0 }
    END {
      printf "ip_link: RXerr=%d RXdrop=%d RXoverrun=%d TXerr=%d TXdrop=%d TXcarrier=%d\n",
             rx_err, rx_drop, rx_over, tx_err, tx_drop, tx_car
    }'
}

check_ethtool() {
  # ALWAYS print selected keys (even if 0) so before/after diff is meaningful.
  local IFACE="$1"
  local BASE="${IFACE%%.*}"

  # Try iface then base iface for VLAN subifaces
  local OUT
  OUT="$(sudo ethtool -S "$IFACE" 2>/dev/null || sudo ethtool -S "$BASE" 2>/dev/null || true)"

  if [[ -z "$OUT" ]]; then
    echo "ethtool: (no stats available)"
    return 0
  fi

  # Keys are NIC/driver dependent; we print a curated set if present.
  # If a key doesn't exist on this NIC, it won't print (that's fine).
  echo "$OUT" | awk -F':' '
    BEGIN {
      # list of high-signal counters you usually want
      want["rx_out_of_buffer"]=1
      want["rx_missed_errors"]=1
      want["rx_dropped"]=1
      want["tx_dropped"]=1
      want["rx_errors"]=1
      want["tx_errors"]=1
      want["rx_discards_phy"]=1
      # also match rx_prioX_discards if present
    }
    {
      k=$1; v=$2
      gsub(/^[ \t]+|[ \t]+$/, "", k)
      gsub(/^[ \t]+|[ \t]+$/, "", v)
      lk=tolower(k)

      if (want[lk]) {
        printf "ethtool: %s %d\n", lk, (v+0)
      } else if (lk ~ /^rx_prio[0-9]+_discards$/) {
        printf "ethtool: %s %d\n", lk, (v+0)
      }
    }'
}

check_tc_qdisc_class() {
  # ALWAYS print dropped number if present; if no qdisc/class or no "Sent" lines, prints nothing (OK).
  local IFACE="$1"

  sudo tc -s qdisc show dev "$IFACE" 2>/dev/null | \
    awk '
      /Sent/ {
        d=0
        if (match($0,/dropped +([0-9]+)/,m)) d=m[1]+0
        printf "qdisc: %s dropped=%d\n", $0, d
      }'

  sudo tc -s class show dev "$IFACE" 2>/dev/null | \
    awk '
      /Sent/ {
        d=0
        if (match($0,/dropped +([0-9]+)/,m)) d=m[1]+0
        printf "class: %s dropped=%d\n", $0, d
      }'
}

check_softnet_stat() {
  # ALWAYS print per-CPU drops and also sum.
  local i=0
  local sum=0
  while read -r line; do
    i=$((i+1)); set -- $line
    drops=$((16#$2))
    sum=$((sum + drops))
    echo "softnet: CPU$((i-1)) backlog_drops_total=$drops"
  done < /proc/net/softnet_stat
  echo "softnet: backlog_drops_sum=$sum"
}

check_nstat() {
  # ALWAYS print these counters (even if 0) when nstat exists.
  if ! command -v nstat >/dev/null 2>&1; then
    echo "nstat: (not installed)"
    return 0
  fi
  nstat -az 2>/dev/null | awk '
    BEGIN {
      want["IpInDiscards"]=1
      want["IpInErrors"]=1
      want["IpOutDiscards"]=1
      want["IpOutNoRoutes"]=1
      want["TcpInErrs"]=1
      want["TcpRetransSegs"]=1
      want["TcpOutRsts"]=1
    }
    ($1 in want) { printf "nstat: %s = %s\n", $1, $2 }
  '
}

check_netstat_ext() {
  # TcpExt/IpExt are in /proc/net/netstat. We print a small set if present.
  # Output format is stable: a "TcpExt:" header line followed by values.
  python3 - <<'PY' 2>/dev/null || true
import re

want_tcp = {
  "TCPBacklogDrop",
  "TCPRcvQDrop",
  "TCPTimeouts",
  "TCPAbortOnTimeout",
  "TCPSynRetrans",
  "TCPFastRetrans",
  "TCPLossProbes",
}
want_ip = {
  "InDiscards",
  "InNoRoutes",
  "InTruncatedPkts",
  "OutDiscards",
}

txt = open("/proc/net/netstat","r").read().splitlines()
def parse_section(prefix):
  for i in range(len(txt)-1):
    if txt[i].startswith(prefix) and txt[i+1].startswith(prefix):
      keys = txt[i].split()[1:]
      vals = txt[i+1].split()[1:]
      m = {}
      for k,v in zip(keys, vals):
        try: m[k]=int(v)
        except: pass
      return m
  return {}

tcp = parse_section("TcpExt:")
ipx = parse_section("IpExt:")

# Print only if key exists (but ALWAYS print value when key exists, even if 0)
for k in sorted(want_tcp):
  if k in tcp:
    print(f"tcpext: {k} = {tcp[k]}")
for k in sorted(want_ip):
  if k in ipx:
    print(f"ipext: {k} = {ipx[k]}")
PY
}

check_ss_summary() {
  # Host-wide socket summary (cheap and useful)
  if command -v ss >/dev/null 2>&1; then
    ss -s 2>/dev/null | sed 's/^/ss: /'
  else
    echo "ss: (not installed)"
  fi
}

# --- Run per-interface checks ---
for IFACE in "${IFACES[@]}"; do
  echo "=== Checking $IFACE ==="
  OUT=$(
    check_ip_link "$IFACE"
    check_ethtool "$IFACE"
    check_tc_qdisc_class "$IFACE"
  )
  echo "$OUT"

  # "clean" means: all the printed counters are zero (excluding lines like "ethtool: (no stats available)")
  if echo "$OUT" | egrep -q '=(0|0\n)|\s0$'; then
    : # keep going; we decide ok/rc below
  fi

  # Set ok=false if any obvious nonzero appears
  if echo "$OUT" | egrep -q '(=([1-9][0-9]*)\b)|(\s([1-9][0-9]*)$)|dropped=[1-9]'; then
    ok=false
  fi
  echo
done

# --- System-wide checks ---
echo "=== Checking softnet backlog drops (per-CPU) ==="
SN=$(check_softnet_stat)
echo "$SN"
if echo "$SN" | egrep -q 'backlog_drops_(total|sum)=[1-9]'; then
  ok=false
fi
echo

echo "=== Checking nstat (TCP/IP stack counters) ==="
NS=$(check_nstat)
echo "$NS"
if echo "$NS" | egrep -q '= [1-9]'; then
  ok=false
fi
echo

echo "=== Checking /proc/net/netstat (TcpExt/IpExt) ==="
EXT=$(check_netstat_ext)
if [[ -z "$EXT" ]]; then
  echo "netstat_ext: (no matching keys found on this kernel)"
else
  echo "$EXT"
  if echo "$EXT" | egrep -q '= [1-9]'; then
    ok=false
  fi
fi
echo

echo "=== Checking ss -s ==="
check_ss_summary
echo

# --- Result ---
if $ok; then
  echo "✅✅ All counters look clean (all zeros for tracked fields)."
  rc=0
else
  echo "⚠️ Nonzero counters detected above."
  rc=1
fi

echo "=== Drop Check Finished at $(date) ==="
echo "Results saved to $LOGFILE"
exit $rc
'''
    return prefix + body


In [ ]:
def run_drop_check(node, expname: str):
    """Execute the drop-check on a given node with a chosen experiment name."""
    cmd = build_drop_check_command(expname)
    stdout, stderr = node.execute(cmd, quiet=True)

In [ ]:
import re

METRICS = [
    "rx_out_of_buffer",
    "rx_discards_phy",
    "rx_prio_discards",
    "ip_rxdrop",
    "ip_txdrop",
    "tc_qdisc_drop",
    "tc_class_drop",
    "softnet_backlog_drops_sum",
    "retrans",
    "tcpext_TCPBacklogDrop",
    "tcpext_TCPRcvQDrop",
    "tcpext_TCPTimeouts",
    "tcpext_TCPAbortOnTimeout",
    "rx_missed_errors",
    "rx_dropped",
    "tx_dropped",
    "rx_errors",
    "tx_errors"
]



def parse_drop_log(log_text: str) -> dict:
    iface_counters = {}  # iface -> metric -> value
    current_iface = None

    host_retrans = 0
    host_softnet_sum = 0
    host_tcpext = {}

    def ensure_iface(iface):
        if iface not in iface_counters:
            iface_counters[iface] = {
                "rx_out_of_buffer": 0,
                "rx_discards_phy": 0,
                "rx_prio_discards": 0,
                "ip_rxdrop": 0,
                "ip_txdrop": 0,
                "tc_qdisc_drop": 0,
                "tc_class_drop": 0,
                "rx_missed_errors": 0,
                "rx_dropped": 0,
                "tx_dropped": 0,
                "rx_errors": 0,
                "tx_errors": 0,
            }

    for raw in log_text.splitlines():
        line = raw.strip()

        # -------------------------
        # Host-wide parsers first
        # -------------------------

        m = re.match(r"softnet:\s+backlog_drops_sum=(\d+)", line)
        if m:
            host_softnet_sum = int(m.group(1))
            continue

        m = re.match(r"nstat:\s+TcpRetransSegs\s+=\s+(\d+)", line)
        if m:
            host_retrans = int(m.group(1))
            continue

        m = re.match(r"tcpext:\s+(\S+)\s+=\s+(\d+)", line)
        if m:
            host_tcpext[m.group(1)] = int(m.group(2))
            continue

        # -------------------------
        # Interface section switch
        # -------------------------
        m = re.match(r"=== Checking (\S+) ===", line)
        if m:
            current_iface = m.group(1)
            ensure_iface(current_iface)
            continue

        if not current_iface:
            continue

        # -------------------------
        # Per-interface parsers
        # -------------------------

        # ip -s link normalized line
        m = re.match(
            r"ip_link:\s+RXerr=\d+\s+RXdrop=(\d+)\s+RXoverrun=\d+\s+TXerr=\d+\s+TXdrop=(\d+)\s+TXcarrier=\d+",
            line
        )
        if m:
            iface_counters[current_iface]["ip_rxdrop"] = int(m.group(1))
            iface_counters[current_iface]["ip_txdrop"] = int(m.group(2))
            continue

        # ethtool lines: "ethtool: <key> <val>"
        m = re.match(r"ethtool:\s+(\S+)\s+(\d+)", line)
        if m:
            key = m.group(1)
            val = int(m.group(2))
            ensure_iface(current_iface)

            if key in (
                "rx_out_of_buffer",
                "rx_discards_phy",
                "rx_missed_errors",
                "rx_dropped",
                "tx_dropped",
                "rx_errors",
                "tx_errors",
            ):
                iface_counters[current_iface][key] = val
                continue

            if re.match(r"rx_prio\d+_discards", key):
                iface_counters[current_iface]["rx_prio_discards"] += val
                continue

            continue

        # tc qdisc/class: dropped=<num>
        m = re.search(r"qdisc: .*dropped=(\d+)", line)
        if m:
            iface_counters[current_iface]["tc_qdisc_drop"] += int(m.group(1))
            continue

        m = re.search(r"class: .*dropped=(\d+)", line)
        if m:
            iface_counters[current_iface]["tc_class_drop"] += int(m.group(1))
            continue

    # --- Group by base interface (avoid VLAN double count) ---
    grouped = {}
    for iface, stats in iface_counters.items():
        base = iface.split(".")[0]
        if base not in grouped:
            grouped[base] = {k: 0 for k in stats.keys()}
        for k, v in stats.items():
            grouped[base][k] = max(grouped[base][k], v)

    # --- Sum across base interfaces to host totals ---
    total = {k: 0 for k in METRICS}
    for base, stats in grouped.items():
        for k, v in stats.items():
            if k in total:
                total[k] += v

    # host-wide
    total["retrans"] = host_retrans
    total["softnet_backlog_drops_sum"] = host_softnet_sum

    total["tcpext_TCPBacklogDrop"] = host_tcpext.get("TCPBacklogDrop", 0)
    total["tcpext_TCPRcvQDrop"] = host_tcpext.get("TCPRcvQDrop", 0)
    total["tcpext_TCPTimeouts"] = host_tcpext.get("TCPTimeouts", 0)
    total["tcpext_TCPAbortOnTimeout"] = host_tcpext.get("TCPAbortOnTimeout", 0)

    return total



def print_counter_diff(label: str, before: dict, after: dict):
    print(f"\n=== DIFF for {label} ===")
    for k in METRICS:
        b = before.get(k, 0)
        a = after.get(k, 0)
        diff = a - b
        print(f"{k:18s}: before={b:8d}  after={a:8d}  diff={diff:+d}")
    print("====================================\n")



In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [8],  # n x bandwidth delay product (BDP)
    'btl_capacity': [60], #in Gbps
    'base_rtt': [10], # in ms
    #'aqm': ['FIFO', 'single-queue-FQ', 'pie_drop', 'Codel_drop', 'Codel', 'FQ', 'FQ_Codel', 'FQ_Codel_L4S', 'DualPI2'],
    'aqm': ['DPDK'],
    'ecn_threshold': [60], # in ms
    'ecn': [1],  # 0: noecn, 1: ecn
    'cc': ["bbr1","cubic","bbr3"],
    'flow_number': [10,1], # flow per file
    'duration': [120],
    'file_size':[750], # in GB
    'trial': [1,2,3,4,5,6,7,8,9,10] # it means total 10 trials here.
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing ECN factor from FIFO bottleneck because it does not support ECN. This could be done also for pie_drop and codel_drop but we have not done.
# Removing the cases where ECN Threshold is less than or equal to the buffer size in time, these cases are not meaningful in practice.

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())

        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir = slice_name + '-same-site'

print("Number of experiments:",len(exp_lists))

In [ ]:
for i, s in enumerate(senders, start=1):
    s.execute("sudo modprobe tcp_bbr")
    s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=bbr")
    s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

for i, s in enumerate(receivers, start=1):
    s.execute("sudo modprobe tcp_bbr")
    s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=bbr")
    s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

In [ ]:
senders   = [sender1, sender2, sender3, sender4, sender5, sender6, sender7, sender8]
receivers = [receiver1, receiver2, receiver3, receiver4, receiver5, receiver6, receiver7, receiver8]
inter_routers   = [inter_router1, inter_router2, inter_router3, inter_router4]
routers   = [router1, router2, router3, router4]
end_nodes=senders+receivers

In [ ]:
def run_and_parse(node, prefix, name):
    full = f"{prefix}_{name}"

    before_log = node.execute(f"cat /home/ubuntu/drop_check_{full}_before.log", quiet=True)[0]
    after_log  = node.execute(f"cat /home/ubuntu/drop_check_{full}.log", quiet=True)[0]

    before = parse_drop_log(before_log)
    after  = parse_drop_log(after_log)

    return prefix, before, after   # <-- do NOT print here


In [ ]:
import time

prev_cc="bbr1"

old_flow_number=0

# for exp in exp_lists:

#     if exp['aqm']=="FIFO":
#         exp['ecn_threshold']=0


#     if exp['cc']=="prague":
#         exp['ecn']=3

#     name = str(exp['cc'])+"_" +str(exp['aqm']) +"_" +str(exp['ecn']) +"_" +str(exp['ecn_threshold']) +"_" +str(exp['n_bdp'])+"_"+str(exp['btl_capacity'])+"_"+str(exp['base_rtt'])+"_"+ str(exp['flow_number']) +"_" +str(exp['duration'])+"_"+str(exp['file_size']) +"_"+str(exp['ssthresh_bdp_level'])+"_"+str(exp['reset_timer'])+"_"+str(exp['trial'])
#     name_tx= name+ ".txt"


#     stdout_tx_json, stderr_tx_json = receiver1.execute("ls " + name_tx, quiet=True)


#     # if len(stdout_tx_json):
#     #     print("Already have " + name_tx + ", skipping")

#     # elif len(stderr_tx_json):

#     if stdout_tx_json.strip():
#         print("Already have " + name_tx + ", skipping")
#         continue

#     # ---- FIXED CONDITION ----
#     if str(exp['cc']) in ["bbr3", "bbr"]:
#         alt_cc = "bbr" if str(exp['cc']) == "bbr3" else "bbr3"
#         name_alt = name.replace(str(exp['cc']) + "_", alt_cc + "_", 1)
#         name_tx_alt = name_alt + ".txt"

#         stdout_alt, stderr_alt = receiver1.execute("ls " + name_tx_alt, quiet=True)

#         if stdout_alt.strip():
#             print("Already have " + name_tx_alt + ", skipping")
#             continue

#         print("Running:",exp)

for exp in exp_lists:

    if exp['aqm'] == "FIFO":
        exp['ecn_threshold'] = 0

    if exp['cc'] == "prague":
        exp['ecn'] = 3

    base = (
        str(exp['aqm']) + "_" + str(exp['ecn']) + "_" +
        str(exp['ecn_threshold']) + "_" + str(exp['n_bdp']) + "_" +
        str(exp['btl_capacity']) + "_" + str(exp['base_rtt']) + "_" +
        str(exp['flow_number']) + "_" + str(exp['duration']) + "_" +
        str(exp['file_size']) + "_" + str(exp['ssthresh_bdp_level']) + "_" +
        str(exp['reset_timer']) + "_" + str(exp['trial'])
    )

    # ---- build filenames ----
    names = []

    if exp['cc'] in ["bbr", "bbr3"]:
        names.append("bbr_" + base + ".txt")
        names.append("bbr3_" + base + ".txt")
    else:
        names.append(str(exp['cc']) + "_" + base + ".txt")

    # ---- check existence ----
    found = False
    for name_tx in names:
        stdout, _ = receiver1.execute("ls " + name_tx, quiet=True)
        if stdout.strip():
            print("Already have " + name_tx + ", skipping")
            found = True
            break

    if found:
        continue

    if not found:

        name=str(exp['cc']) + "_" + base

        print("Running:", exp)

        if exp['cc'] =="bbr3":

            exp['cc'] = "bbr"

        if exp["cc"]!=prev_cc:

            for i, s in enumerate(senders, start=1):
                s.execute("sudo modprobe tcp_"+exp["cc"])
                s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control="+exp["cc"])
                s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

            for i, s in enumerate(receivers, start=1):
                s.execute("sudo modprobe tcp_"+exp["cc"])
                s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control="+exp["cc"])
                s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

            prev_cc= exp["cc"]



        # for idx, s in enumerate(senders, start=1):
        #     out = s.execute("ethtool -S enp7s0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"sender{idx}: {out}")
        #     _ = run_drop_check(s, f"sender{idx}_{name}_before")

        # for idx, r in enumerate(receivers, start=1):
        #     out = r.execute("ethtool -S enp7s0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"receiver{idx}: {out}")
        #     _ = run_drop_check(r, f"receiver{idx}_{name}_before")

        # for idx, r in enumerate(routers, start=1):
        #     out = r.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"router{idx}: {out}")
        #     _ = run_drop_check(r, f"router{idx}_{name}_before")

        # if exp['aqm']=="linux" or exp['aqm']=="DPDK":

        #     for idx, r in enumerate(inter_routers, start=1):
        #         out = r.execute("ethtool -S enp7s0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #         print(f"inter_router{idx}: {out}")
        #         _ = run_drop_check(r, f"inter_router{idx}_{name}_before")


        receiver3.execute("sudo killall iperf3")
        sender3.execute("sudo killall iperf3")

        receiver4.execute("sudo killall iperf3")
        sender4.execute("sudo killall iperf3")

        receiver1.execute("sudo killall iperf3")
        sender1.execute("sudo killall iperf3")


        receiver2.execute("sudo killall iperf3")
        sender2.execute("sudo killall iperf3")


        receiver5.execute("sudo killall iperf3")
        sender5.execute("sudo killall iperf3")

        receiver6.execute("sudo killall iperf3")
        sender6.execute("sudo killall iperf3")

        receiver7.execute("sudo killall iperf3")
        sender7.execute("sudo killall iperf3")

        receiver8.execute("sudo killall iperf3")
        sender8.execute("sudo killall iperf3")


        time.sleep(5)

        #user_01_tm.cfg.04f.flat


        if exp['aqm']=="DPDK":

            print("Running DPDK")

            inter_router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            inter_router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")


            time.sleep(1)


            inter_router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            inter_router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")


            time.sleep(1)


            inter_router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            inter_router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")


            time.sleep(1)


            inter_router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            inter_router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")


            # time.sleep(1)


            router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(100*1000)} user_01_tm01_1flow.cfg 1 > output-{name}.log 2>&1")
            #router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            #router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(100*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")
            #router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")


            #router4.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10-tm10-rev/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int((exp['btl_capacity'])*1000)} user_01_tm.cfg.04f.flat 1 > output-reverse-{name}.log 2>&1")

            time.sleep(1)


            #router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(100*1000)} user_01_tm01_1flow.cfg 1 > output-{name}.log 2>&1")
            router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            #router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(100*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")
            router2.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")

            time.sleep(1)



            #router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(100*1000)} user_01_tm01_1flow.cfg 1 > output-{name}.log 2>&1")
            router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            #router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(100*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")
            router1.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")

            time.sleep(1)


            router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(100*1000)} user_01_tm01_1flow.cfg 1 > output-{name}.log 2>&1")
            #router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            #router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm.cfg.04f.flat 1 > output-{name}.log 2>&1")

            time.sleep(1)

            router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(100*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")
            #router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")

            #router3.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10-tm10-rev/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int((exp['btl_capacity'])*1000)} user_01_tm.cfg.04f.flat 1 > output-reverse-{name}.log 2>&1")

            time.sleep(1)


        receiver3.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver4.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver1.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver2.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver5.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver6.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")
        receiver7.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

        receiver8.execute_thread(f"iperf3 -s -1 -A 6-15 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")




        time.sleep(10)




        # sender3.execute_thread(f"sleep 1; iperf3 -Z -c {receiver3_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender4.execute_thread(f"sleep 1; iperf3 -Z -c {receiver4_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender1.execute_thread(f"sleep 1; iperf3 -Z -c {receiver1_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender2.execute_thread(f"sleep 1; iperf3 -Z -c {receiver2_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender5.execute_thread(f"sleep 1; iperf3 -Z -c {receiver5_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender6.execute_thread(f"sleep 1; iperf3 -Z -c {receiver6_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender7.execute_thread(f"sleep 1; iperf3 -Z -c {receiver7_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        # sender8.execute_thread(f"sleep 1; iperf3 -Z -c {receiver8_addr} -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) + " -A 6-15" +" -J > "+ name+".json")


        #filesize=int(((exp['btl_capacity']/4)*50)/4)
        handles = []
        #for i in [1,3,5,7]:
        for i in range(1,9):
            sender = globals()[f"sender{i}"]
            receiver_addr = globals()[f"receiver{i}_addr"]

            h = sender.execute_thread(
                f"iperf3 -Z -c {receiver_addr} -p 33000 --cport 30001 -n "
                + str(exp['file_size']) + "G"
                + " -C " + exp["cc"]
                + " -P " + str(exp['flow_number'])
                + " -A 6-15"
                + " -J > " + name + ".json"
            )
            # h = sender.execute_thread(
            #     f"iperf3 -Z -c {receiver_addr} -p 33000 --cport 30001 -n "
            #     + str(filesize) + "G"
            #     + " -C " + exp["cc"]
            #     + " -P " + str(exp['flow_number'])
            #     + " -A 6-15"
            #     + " -J > " + name + ".json"
            # )
            handles.append(h)
            time.sleep(1)


        # wait for all
        for h in handles:
            h.result()


        time.sleep(5)


        #time.sleep(exp['duration'] + 10)


        if exp['aqm']=="DPDK":

            print("Stopping DPDK")


            inter_router2.execute("sudo pkill -f tm10")
            inter_router2.execute("pkill -f 'mpstat -P ALL 1'")

            inter_router2.execute("sudo rm -f /mnt/huge/rte*map_*")

            inter_router2.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            inter_router2.execute("sudo rm -f /dev/hugepages/rte*map_*")


            inter_router1.execute("sudo pkill -f tm10")
            inter_router1.execute("pkill -f 'mpstat -P ALL 1'")

            inter_router1.execute("sudo rm -f /mnt/huge/rte*map_*")

            inter_router1.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            inter_router1.execute("sudo rm -f /dev/hugepages/rte*map_*")



            inter_router3.execute("sudo pkill -f tm10")
            inter_router3.execute("pkill -f 'mpstat -P ALL 1'")

            inter_router3.execute("sudo rm -f /mnt/huge/rte*map_*")

            inter_router3.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            inter_router3.execute("sudo rm -f /dev/hugepages/rte*map_*")


            inter_router4.execute("sudo pkill -f tm10")
            inter_router4.execute("pkill -f 'mpstat -P ALL 1'")

            inter_router4.execute("sudo rm -f /mnt/huge/rte*map_*")

            inter_router4.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            inter_router4.execute("sudo rm -f /dev/hugepages/rte*map_*")


            router4.execute("sudo pkill -f tm10")
            router4.execute("pkill -f 'mpstat -P ALL 1'")

            router4.execute("sudo rm -f /mnt/huge/rte*map_*")

            router4.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            router4.execute("sudo rm -f /dev/hugepages/rte*map_*")


            router2.execute("sudo pkill -f tm10")
            router2.execute("pkill -f 'mpstat -P ALL 1'")

            router2.execute("sudo rm -f /mnt/huge/rte*map_*")

            router2.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            router2.execute("sudo rm -f /dev/hugepages/rte*map_*")


            router1.execute("sudo pkill -f tm10")
            router1.execute("pkill -f 'mpstat -P ALL 1'")

            router1.execute("sudo rm -f /mnt/huge/rte*map_*")

            router1.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            router1.execute("sudo rm -f /dev/hugepages/rte*map_*")


            router3.execute("sudo pkill -f tm10")
            router3.execute("pkill -f 'mpstat -P ALL 1'")

            router3.execute("sudo rm -f /mnt/huge/rte*map_*")

            router3.execute("sudo rm -f /mnt/huge1G/rte*map_*")

            router3.execute("sudo rm -f /dev/hugepages/rte*map_*")


        # router2.execute("sudo pkill -f tm10")
        # router2.execute("pkill -f 'mpstat -P ALL 1'")

        # router2.execute("sudo rm -f /mnt/huge/rte*map_*")

        # router2.execute("sudo rm -f /mnt/huge1G/rte*map_*")

        # router2.execute("sudo rm -f /dev/hugepages/rte*map_*")


        # router3.execute("sudo pkill -f tm10")
        # router3.execute("pkill -f 'mpstat -P ALL 1'")

        # router3.execute("sudo rm -f /mnt/huge/rte*map_*")

        # router3.execute("sudo rm -f /mnt/huge1G/rte*map_*")

        # router3.execute("sudo rm -f /dev/hugepages/rte*map_*")


        # router4.execute("sudo pkill -f tm10")
        # router4.execute("pkill -f 'mpstat -P ALL 1'")

        # router4.execute("sudo rm -f /mnt/huge/rte*map_*")

        # router4.execute("sudo rm -f /mnt/huge1G/rte*map_*")

        # router4.execute("sudo rm -f /dev/hugepages/rte*map_*")


        receiver3.execute("sudo killall iperf3")
        sender3.execute("sudo killall iperf3")

        receiver4.execute("sudo killall iperf3")
        sender4.execute("sudo killall iperf3")

        receiver1.execute("sudo killall iperf3")
        sender1.execute("sudo killall iperf3")


        receiver2.execute("sudo killall iperf3")
        sender2.execute("sudo killall iperf3")


        receiver5.execute("sudo killall iperf3")
        sender5.execute("sudo killall iperf3")

        receiver6.execute("sudo killall iperf3")
        sender6.execute("sudo killall iperf3")

        receiver7.execute("sudo killall iperf3")
        sender7.execute("sudo killall iperf3")

        receiver8.execute("sudo killall iperf3")
        sender8.execute("sudo killall iperf3")

        # for idx, s in enumerate(senders, start=1):
        #     out = s.execute("ethtool -S enp7s0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"sender{idx}: {out}")
        #     _ = run_drop_check(s, f"sender{idx}_{name}")

        # for idx, r in enumerate(receivers, start=1):
        #     out = r.execute("ethtool -S enp7s0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"receiver{idx}: {out}")
        #     _ = run_drop_check(r, f"receiver{idx}_{name}")

        # for idx, r in enumerate(routers, start=1):
        #     out = r.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #     print(f"router{idx}: {out}")
        #     _ = run_drop_check(r, f"router{idx}_{name}")

        # if exp['aqm']=="linux" or exp['aqm']=="DPDK":

        #     for idx, r in enumerate(inter_routers, start=1):
        #         out = r.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer", quiet=True)[0].strip()
        #         print(f"inter_router{idx}: {out}")
        #         _ = run_drop_check(r, f"inter_router{idx}_{name}")


        # results = []

        # for i, s in enumerate(senders, start=1):
        #     print(f"sender{i} results are collected.")
        #     results.append(run_and_parse(s, f"sender{i}", name))

        # for i, r in enumerate(receivers, start=1):
        #     print(f"receiver{i} results are collected.")
        #     results.append(run_and_parse(r, f"receiver{i}", name))

        # for i, rt in enumerate(routers, start=1):
        #     print(f"router{i} results are collected.")
        #     results.append(run_and_parse(rt, f"router{i}", name))

        # if exp['aqm']=="linux" or exp['aqm']=="DPDK":

        #     for i, rt in enumerate(inter_routers, start=1):
        #         print(f"inter_router{i} results are collected.")
        #         results.append(run_and_parse(rt, f"inter_router{i}", name))


        # print("\n==== DROP SUMMARY ====\n")

        # for prefix, before, after in results:
        #     print_counter_diff(prefix, before, after)

        print("###################### GOODPUTS############################")

        for i, n in enumerate(receivers):
            print(f"receiver{i+1}")
            n.execute(f"tail -n 1 {name}.txt")

        time.sleep(25)

print("done")


#output = receiver_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
#print(f"receiver enp7s0np0: {output}")

#output = sender_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
#print(f"sender enp7s0np0: {output}")

In [ ]:
inter_router2.execute("sudo pkill -f tm10")
inter_router2.execute("pkill -f 'mpstat -P ALL 1'")

inter_router2.execute("sudo rm -f /mnt/huge/rte*map_*")

inter_router2.execute("sudo rm -f /mnt/huge1G/rte*map_*")

inter_router2.execute("sudo rm -f /dev/hugepages/rte*map_*")


inter_router1.execute("sudo pkill -f tm10")
inter_router1.execute("pkill -f 'mpstat -P ALL 1'")

inter_router1.execute("sudo rm -f /mnt/huge/rte*map_*")

inter_router1.execute("sudo rm -f /mnt/huge1G/rte*map_*")

inter_router1.execute("sudo rm -f /dev/hugepages/rte*map_*")



inter_router3.execute("sudo pkill -f tm10")
inter_router3.execute("pkill -f 'mpstat -P ALL 1'")

inter_router3.execute("sudo rm -f /mnt/huge/rte*map_*")

inter_router3.execute("sudo rm -f /mnt/huge1G/rte*map_*")

inter_router3.execute("sudo rm -f /dev/hugepages/rte*map_*")


inter_router4.execute("sudo pkill -f tm10")
inter_router4.execute("pkill -f 'mpstat -P ALL 1'")

inter_router4.execute("sudo rm -f /mnt/huge/rte*map_*")

inter_router4.execute("sudo rm -f /mnt/huge1G/rte*map_*")

inter_router4.execute("sudo rm -f /dev/hugepages/rte*map_*")

In [ ]:
router4.execute("sudo pkill -f tm10")
router4.execute("pkill -f 'mpstat -P ALL 1'")

router4.execute("sudo rm -f /mnt/huge/rte*map_*")

router4.execute("sudo rm -f /mnt/huge1G/rte*map_*")

router4.execute("sudo rm -f /dev/hugepages/rte*map_*")


router2.execute("sudo pkill -f tm10")
router2.execute("pkill -f 'mpstat -P ALL 1'")

router2.execute("sudo rm -f /mnt/huge/rte*map_*")

router2.execute("sudo rm -f /mnt/huge1G/rte*map_*")

router2.execute("sudo rm -f /dev/hugepages/rte*map_*")


router1.execute("sudo pkill -f tm10")
router1.execute("pkill -f 'mpstat -P ALL 1'")

router1.execute("sudo rm -f /mnt/huge/rte*map_*")

router1.execute("sudo rm -f /mnt/huge1G/rte*map_*")

router1.execute("sudo rm -f /dev/hugepages/rte*map_*")


router3.execute("sudo pkill -f tm10")
router3.execute("pkill -f 'mpstat -P ALL 1'")

router3.execute("sudo rm -f /mnt/huge/rte*map_*")

router3.execute("sudo rm -f /mnt/huge1G/rte*map_*")

router3.execute("sudo rm -f /dev/hugepages/rte*map_*")

In [ ]:
receiver3.execute("sudo killall iperf3")
sender3.execute("sudo killall iperf3")

receiver4.execute("sudo killall iperf3")
sender4.execute("sudo killall iperf3")

receiver1.execute("sudo killall iperf3")
sender1.execute("sudo killall iperf3")


receiver2.execute("sudo killall iperf3")
sender2.execute("sudo killall iperf3")


receiver5.execute("sudo killall iperf3")
sender5.execute("sudo killall iperf3")

receiver6.execute("sudo killall iperf3")
sender6.execute("sudo killall iperf3")

receiver7.execute("sudo killall iperf3")
sender7.execute("sudo killall iperf3")

receiver8.execute("sudo killall iperf3")
sender8.execute("sudo killall iperf3")


In [ ]:
def run_and_parse(node, prefix, name):
    full = f"{prefix}_{name}"

    before_log = node.execute(f"cat /home/ubuntu/drop_check_{full}_before.log", quiet=True)[0]
    after_log  = node.execute(f"cat /home/ubuntu/drop_check_{full}.log", quiet=True)[0]

    before = parse_drop_log(before_log)
    after  = parse_drop_log(after_log)

    return prefix, before, after   # <-- do NOT print here


In [ ]:
for i, s in enumerate(senders, start=1):
    s.execute("sudo modprobe tcp_bbr1")
    s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=bbr1")
    s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

for i, s in enumerate(receivers, start=1):
    s.execute("sudo modprobe tcp_bbr1")
    s.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=bbr1")
    s.execute("sudo sysctl -w net.ipv4.tcp_ecn=1")

### Getting Data

In [ ]:
senders   = [sender1, sender2, sender3, sender4, sender5, sender6, sender7, sender8]
receivers = [receiver1, receiver2, receiver3, receiver4, receiver5, receiver6, receiver7, receiver8]
inter_routers   = [inter_router1, inter_router2, inter_router3, inter_router4]
routers   = [router1, router2, router3, router4]

end_nodes=senders+receivers

In [ ]:
cmds_py_install = '''
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''
for n in end_nodes:
    n.execute(cmds_py_install)

#sender1.execute(cmds_py_install)
#receiver_node.execute(cmds_py_install)

In [ ]:
data_dir="exp-results"


In [ ]:
for n in routers:
    n.execute(f"mkdir -p {data_dir}")
    n.execute(f"mv *.log {data_dir} 2>/dev/null || true")

In [ ]:
for n in senders:
    n.execute(f"mkdir -p {data_dir}")
    #n.execute(f"mv *.log {data_dir} 2>/dev/null || true")
    n.execute(f"mv *.json {data_dir} 2>/dev/null || true")

In [ ]:
for n in receivers:
    n.execute(f"mkdir -p {data_dir}")
    #n.execute(f"mv *.log {data_dir} 2>/dev/null || true")
    n.execute(f"mv *.txt {data_dir} 2>/dev/null || true")

In [ ]:
for n in inter_routers:
    n.execute(f"mkdir -p {data_dir}")
    n.execute(f"mv *.log {data_dir} 2>/dev/null || true")

In [ ]:
for i, n in enumerate(senders, start=1):

    n.upload_file("/home/fabric/work/Internship-work/analysis_final.py", f"/home/ubuntu/{data_dir}/analysis.py")

    n.execute(f'python3 /home/ubuntu/{data_dir}/analysis.py')

    # filename prefix using the sender index
    # outname1 = f"sender{i}-fct-DPDK.json"
    # outname2 = f"sender{i}-retrans-DPDK.json"
    # outname3 = f"sender{i}-rtt-DPDK.json"
    # outname4 = f"sender{i}-received_byte-DPDK.json"

    outname1 = f"sender{i}-fct.json"
    outname2 = f"sender{i}-retrans.json"
    outname3 = f"sender{i}-rtt.json"
    outname4 = f"sender{i}-received_byte.json"


    # remote source stays the same, local filename changes
    n.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{outname1}", f"/home/ubuntu/{data_dir}/fct_data.json")
    n.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{outname2}",f"/home/ubuntu/{data_dir}/retransmits_data.json")
    n.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{outname3}",f"/home/ubuntu/{data_dir}/stream_rtt_data.json")
    n.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{outname4}",f"/home/ubuntu/{data_dir}/received_byte.json")

In [ ]:
import json

for i, n in enumerate(senders, start=1):


     # Load both JSONs
    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-received_byte.json") as f:
        results_byte = json.load(f)

    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-fct.json") as f:
        results_fct = json.load(f)

    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-retrans.json") as f:
        retrans = json.load(f)


    ratios = {}

    ideal_times ={}

    overhead={}

    for name, byte in results_byte.items():
        parts = name.split("_")

        if parts[-1] in {"1", "2", "3", "4", "5"}:

            rate_gbps = float(parts[5])/4

            if parts[1]=="linux":

                processed = ((byte * 8) / (rate_gbps * 1e9)) * (9018 / 8948)

            elif parts[1]=="DPDK":

                processed = ((byte * 8) / (rate_gbps * 0.99 * 1e9)) * (9042 / 8948)

            ideal_times[name] = processed

            other_value = results_fct.get(name)

            if other_value is not None and processed != 0:
                ratios[name] = processed/ other_value

            overhead[name]=(retrans.get(name)*8948*8)/(results_fct.get(name))

    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-ratio.json", "w") as f:
      json.dump(ratios, f, indent=4)


    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-ideal.json", "w") as f:
      json.dump(ideal_times, f, indent=4)

    with open(f"/home/fabric/work/Internship-work/L2P2P-QoS/sender{i}-overhead.json", "w") as f:
      json.dump(overhead, f, indent=4)



### Example Plot

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.ticker import FixedLocator, FixedFormatter, AutoMinorLocator
import matplotlib
from matplotlib.ticker import FormatStrFormatter

# ---------------- visual defaults ----------------
matplotlib.rcParams.update({
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "font.size": 18,
    "axes.titlesize": 20,
    "axes.labelsize": 18,
    "legend.fontsize": 16,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "axes.linewidth": 1.4,
})

XTICK_SIZE = 16
YTICK_SIZE = 16
ANNOTATION_FONTSIZE = 16
BAR_EDGE_LW = 1.2
ERR_LW = 1.2
ERR_CAPSIZE = 5
BAR_GROUP_SPAN = 0.92


def load_metric(path, cc_prefixes, cond_fn, metric_scale=1.0, expected_tag="DPDK"):
    """
    Load values keyed by flow-level (lvl) from JSON dict: name -> value.
    - cc_prefixes: {"bbr1"} or {"bbr3","bbr"}
    - expected_tag: parts[1] must match (e.g., "DPDK")
    - metric_scale: multiply raw value by this (e.g., /1e6 => scale=1e-6)
    Returns: dict seconds[level] -> list(values)
    """
    cc_prefixes = set(cc_prefixes)

    with open(path) as f:
        results = json.load(f)

    values = defaultdict(list)
    for name, v in results.items():
        parts = name.split("_")
        if len(parts) < 7:
            continue
        if parts[0] in cc_prefixes and parts[1] == expected_tag and cond_fn(parts):
            lvl = int(parts[-6])  # flow_number
            values[lvl].append(float(v) * metric_scale)
    return values


def compute_stats_simple(values_by_lvl):
    """Return stats[level] -> (mean, vmin, vmax) using min–max whiskers."""
    return {
        lvl: (np.mean(vals), np.min(vals), np.max(vals))
        for lvl, vals in values_by_lvl.items()
    }


def build_datasets(files_and_conds, cc_prefixes, metric_scale=1.0, expected_tag="DPDK"):
    datasets = []
    all_levels = set()

    for path, cond_fn, label in files_and_conds:
        vals = load_metric(path, cc_prefixes, cond_fn, metric_scale=metric_scale, expected_tag=expected_tag)
        if not vals:
            continue
        stats = compute_stats_simple(vals)
        datasets.append((label, stats))
        all_levels.update(stats.keys())

    return datasets, sorted(all_levels)


def plot_panel(ax, datasets, levels_sorted, sender_colors, title,
               ymin, ymax, value_fmt="{:.3f}", yticks=None, yticklabels=None,
               xlabel=True):
    n_senders = len(sender_colors)
    x = np.arange(len(levels_sorted))
    bar_w = BAR_GROUP_SPAN / n_senders

    ax.set_ylim(ymin, ymax)

    for s_idx, (label, stats) in enumerate(datasets):
        color = sender_colors[s_idx]

        for j, lvl in enumerate(levels_sorted):
            if lvl not in stats:
                continue

            mean, vmin, vmax = stats[lvl]
            xpos = x[j] - BAR_GROUP_SPAN/2 + bar_w/2 + s_idx * bar_w

            ax.bar(
                xpos, mean, bar_w,
                yerr=[[mean - vmin], [vmax - mean]],
                error_kw={"elinewidth": ERR_LW, "capthick": ERR_LW, "capsize": ERR_CAPSIZE},
                color=color,
                edgecolor="black",
                linewidth=BAR_EDGE_LW,
                label=label if j == 0 else None,
            )

            # Mean label (avoid clipping)
            pad = 0.02 * (ymax - ymin)
            if mean > ymin + 3 * pad:
                y_text, va = mean - pad, "top"
            else:
                y_text, va = mean + pad, "bottom"

            # ax.text(
            #     xpos, y_text, f"{mean:.3f}",
            #     ha="center", va=va,
            #     fontsize=ANNOTATION_FONTSIZE,
            #     fontweight="bold",
            #     color="black",
            #     clip_on=False,
            # )

            ax.text(
                xpos, y_text, value_fmt.format(mean),
                ha="center", va=va,
                fontsize=ANNOTATION_FONTSIZE,
                fontweight="bold",
                color="black",
                clip_on=False,
            )

    ax.set_xticks(x)
    ax.set_xticklabels([str(int(l)) for l in levels_sorted])
    if xlabel:
        ax.set_xlabel("Number of flows")
    else:
        ax.set_xlabel("")

    ax.set_title(title, pad=10)

    # y ticks (optional)
    if yticks is not None:
        ax.yaxis.set_major_locator(FixedLocator(yticks))
        if yticklabels is not None:
            ax.yaxis.set_major_formatter(FixedFormatter(yticklabels))

    ax.yaxis.set_minor_locator(AutoMinorLocator(4))
    ax.grid(axis="y", which="major", linestyle="--", alpha=0.45)
    ax.grid(axis="y", which="minor", linestyle=":", alpha=0.25)

    ax.tick_params(axis="x", labelsize=XTICK_SIZE, length=6, width=1.2)
    ax.tick_params(axis="y", labelsize=YTICK_SIZE, length=6, width=1.2)


# ---------- inputs ----------
files_ratio = [
    (f"/content/dpdk-linux/sender{i}-ratio.json", lambda p: True, f"sender{i}") # adjust the location of the files..
    for i in range(1, 9)
]
files_overhead = [
    (f"/content/dpdk-linux/sender{i}-overhead.json", lambda p: True, f"sender{i}") # adjust the location of the files..
    for i in range(1, 9)
]

# Similar 8 shades (single-hue)
n_senders = 8
cmap = plt.get_cmap("Blues")
shade_levels = np.linspace(0.15, 0.75, n_senders)
sender_colors = [cmap(v) for v in shade_levels]

# ---------- build datasets ----------
# Top row: ratio (no scaling)
ratio_v1, lvl_r1 = build_datasets(files_ratio, {"bbr1"}, metric_scale=100)
ratio_v3, lvl_r3 = build_datasets(files_ratio, {"bbr3", "bbr"}, metric_scale=100)

# Bottom row: overhead (convert to Mb/s if your file is in bps: /1e6)
over_v1, lvl_o1 = build_datasets(files_overhead, {"bbr1"}, metric_scale=(1e-6)*(1/20000)*(100))
over_v3, lvl_o3 = build_datasets(files_overhead, {"bbr3", "bbr"}, metric_scale=(1e-6)*(1/20000)*(100))

# Union of levels across everything so x positions match everywhere
levels_sorted = sorted(set(lvl_r1) | set(lvl_r3) | set(lvl_o1) | set(lvl_o3))
if not levels_sorted:
    raise RuntimeError("No flow levels found after filtering.")

# ---------- plot: 2x2 ----------
fig, axes = plt.subplots(
    2, 2,
    figsize=(34, 7),
    sharey=False,
    gridspec_kw={"wspace": 0.05, "hspace": 0.30}
)

# --- top row: ratio ---
plot_panel(
    axes[0, 0], ratio_v1, levels_sorted, sender_colors,
    title="BBRv1 – DPDK shaping (20 Gb/s • 1 TB)",
    ymin=60, ymax=100,
    yticks=[60, 80, 100], yticklabels=["60", "80", "100"],
    xlabel=False,
    value_fmt="{:.1f}"
)

plot_panel(
    axes[0, 1], ratio_v3, levels_sorted, sender_colors,
    title="BBRv3 – DPDK shaping (20 Gb/s • 1 TB)",
    ymin=60, ymax=100,
    yticks=[60, 80, 100], yticklabels=["60", "80", "100"],
    xlabel=False,
    value_fmt="{:.1f}"
)

axes[0, 0].set_ylabel("FCT efficiency [%]")

# --- bottom row: overhead ---
plot_panel(
    axes[1, 0], over_v1, levels_sorted, sender_colors,
    title="",
    ymin=0, ymax=20,
    xlabel=True,
    value_fmt="{:.1f}"
)

plot_panel(
    axes[1, 1], over_v3, levels_sorted, sender_colors,
    title="",
    ymin=0, ymax=20,
    xlabel=True,
    value_fmt="{:.1f}"
)

axes[1, 0].set_ylabel("Overhead (%)")

# ---------- shared legend (right side, moved up) ----------
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(
    handles, labels,
    loc="upper left",
    bbox_to_anchor=(0.905, 0.8),
    frameon=True
)

axes[0, 1].tick_params(axis="y", labelleft=False)
axes[1, 1].tick_params(axis="y", labelleft=False)


# ---------- save ----------
out_pdf = "bbrv1_vs_bbrv3_dpdk_shaping_2x2.pdf"
out_png = "bbrv1_vs_bbrv3_dpdk_shaping_2x2.png"
fig.savefig(out_pdf, bbox_inches="tight")
fig.savefig(out_png, dpi=300, bbox_inches="tight")
plt.show()


### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()